# tootltips

In [ ]:
!curl http://localhost:9200/dpe_pre_1948_td005/_count

# Imports

In [693]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk,parallel_bulk
import time

import subprocess

In [694]:
import os

In [692]:
import subprocess
es_bat=Path(os.environ['USERPROFILE']+r'\apps\elasticsearch-7.9.1-windows-x86_64\elasticsearch-7.9.1\bin\elasticsearch.bat')
p=subprocess.Popen(str(es_bat.absolute()))

In [ ]:
p

# Preparer la base de données pour l'analyse au moment de la sauvegarde

In [ ]:
def clean_desc_txt(x):
    bad_chars_to_space = ['"', "'", "''", '""',  # these char in addr field generate errors on addok
                      ",",  # removed by precaution since we dump comma sep csv and we dont want quotechar
                      "\\n", "\\t","\\r","\\r\\n"  #
                      "\n", "\t",  # remove carriage return/tab
                      "/", "\\", "(", ")"  # remove special separators characters and ()
                                      "[", "]", "_", "°", "^","<br>", ">", "<"
                                                                    '«', '»',

                      ]
    for bad_char in bad_chars_to_space:
        x=x.replace(bad_char,' ')
    return x

In [ ]:
def setup_es_client(index_name):
    es_client = Elasticsearch()

    index_name = index_name
    index_configurations = {
        "settings": {
            "index": {
                "number_of_shards": 4,
                "number_of_replicas": 0,
                "max_result_window": 5000000,

            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "text_to_analyze": {
                    "type": "text",
                    #"analyzer": "french"      
                },
                "category_index":{
                    'type':'keyword'
                }
            }
        }
    }

    try:
        es_client.indices.delete(
            index=index_name,
        )
    except:
        pass
    es_client.indices.create(
        index=index_name,
        body=index_configurations
    )
    return es_client

# fonctions

In [ ]:
import uuid

In [ ]:
from generate_dpe_annexes_scripts.utils import strip_accents

sleepers = {'after_bulk':1,
           'after_search':0.2}
def gendata(index_name,id_data,data):
    for id_,vr in zip(id_data,data):
        yield {
            "_index": index_name,
    "id": id_,
    "text_to_analyze": vr
        }

def generate_instruction_from_list(char_list):
    
    new_char_list=list()
    for char in char_list:
        if isinstance(char,str):
            new_char_list.append(char)
        elif isinstance(char,tuple):
            new_char_list.append('('+') OR ('.join(char)+')')
            
    search_instruction = '('+') AND ('.join(new_char_list) +')'
    search_instruction = search_instruction.replace('AND (NOT) AND','AND NOT')
    search_instruction = search_instruction.replace('(NOT) AND','NOT')

    return search_instruction

def search_from_search_dict(es_client,search_dict,index_name):
    s_list = list()
    for label,char_list in  search_dict.items():
        time.sleep(sleepers['after_search'])
        search_instruction =generate_instruction_from_list(char_list)
        search_body = {
            "query": {
                "query_string": {
                    "query": search_instruction,
                    "default_field": "text_to_analyze"
                },

            },

        }
        #es_client.count(index=index_name, body=search_body)
        a_dict=es_client.search(index=index_name, body=search_body,size=5000000)

        hits = a_dict['hits']['hits']

        s=pd.Series(index=[el['_source']['id'] for el in hits],dtype='str')
        s[:]=label
        s_list.append(s)
    s_all=pd.concat(s_list)
    return s_all

def search_and_affect(data_to_search,id_col,val_col,search_dict):
    index_name = uuid.uuid4()
    es_client = setup_es_client(index_name)
    L=data_to_search.shape[0]
    bulk(es_client, gendata(index_name,data_to_search[id_col],data_to_search[val_col]))
    count_r = es_client.count(index=index_name)['count']
    
    while L!=count_r:
        time.sleep(0.1)
        count_r = es_client.count(index=index_name)['count']

    res_serie = search_from_search_dict(es_client,search_dict,index_name=index_name)

    res_table=res_serie.to_frame('label')

    res_table.index.name = 'id'
    res_table=res_table.reset_index()
    # destroy index a la fin
    try:
        es_client.indices.delete(
            index=index_name,
        )
    except:
        pass

# #     grp = df_drop.groupby('id').label.apply(lambda x: ' + '.join(sorted(list(set(x))))).reset_index()
# #     m=data.merge(grp,on='id',how='left')

#     m.label=m.label.fillna('indetermine')
    return res_table
def categorize_search_res(table,label_cat=None,category_dict=None):
    
    if category_dict is not None:
        table['category']=table.label.replace(category_dict)
    
    if label_cat is not None:
        table['label']=pd.Categorical(table.label,categories=label_cat,ordered=True)

    #df_drop = df.sort_values('label').drop_duplicates(subset=['id','category'])
    
    table.label = table.label.fillna('indetermine')
    return table

def agg_concat_label_text(table,col_label='label',col_vr='valeur_renseignee'):
    table[[col_label,col_vr]]=table[[col_label,col_vr]].fillna('indetermine')
    agg = table.groupby('td001_dpe_id').agg({

    col_label:lambda x: ' + '.join(sorted(list(set(x)))),
    col_vr:lambda x: ' + '.join(sorted(list(set(x))))
    })

    for col in agg:
        agg[col] =  agg[col].apply(lambda x:' + '.join(sorted(list(set([el for el in x.split(' + ') if el!='indetermine'])))))
        agg[col] = agg[col].replace('','indetermine')
    return agg

# load tables

In [ ]:
from generate_dpe_annexes_scripts.utils import reload_package

In [ ]:
import generate_dpe_annexes_scripts.trtvtables
reload_package(generate_dpe_annexes_scripts.trtvtables)

In [ ]:
import numpy as np

import pandas as pd
from pathlib import Path
import json
from generate_dpe_annexes_scripts.utils import round_float_cols, unique_ordered
from config import paths
from generate_dpe_annexes_scripts.trtvtables import DPETrTvTables

trtv = DPETrTvTables()

if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    for dept_dir in Path(data_dir).iterdir():
        if dept_dir.name=='38':
            print(dept_dir)
            annexe_dept_dir = annexe_dir / dept_dir.name
            annexe_dept_dir.mkdir(exist_ok=True, parents=True)
            # LOAD TABLES
            td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
            td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
            td002 = pd.read_csv(dept_dir/'td002_consommation.csv',dtype=str)
            td016 = pd.read_csv(dept_dir/'td016_facture.csv',dtype=str)

            td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
            td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)
            td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
            td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)
            td011_p = pd.read_csv(annexe_dept_dir / 'td011_installation_chauffage_annexe.csv', dtype=str)

            td012_p = pd.read_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv', dtype=str)
            td014_p = pd.read_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv', dtype=str)
            td001_sys_ch= pd.read_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv', dtype=str)
            td001_sys_ecs= pd.read_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv', dtype=str)

            gen= pd.read_csv(annexe_dept_dir / 'td001_annexe_generale.csv', dtype=str)
            td001_sys_ch.columns = [col.replace('_chauffage','_ch') for col in td001_sys_ch]

            print(dept_dir)
            
            # ENVELOPPE PROCESSING
            td001 =td001.rename(columns={'id':'td001_dpe_id'})
            td011_p['type_installation_ch']=td011_p.tv025_type_installation.replace({'Chauffage Individuel':'individuel',
                                        'Chauffage Collectif':'collectif'})
            break

In [ ]:
td014_p.is_ecs_solaire = td014_p.is_ecs_solaire.replace({'False':False,'True':True})

# dump new

In [ ]:
import main
import generate_dpe_annexes_scripts.conversion_normalisation
import generate_dpe_annexes_scripts.td011_td012_processing
import generate_dpe_annexes_scripts.td013_td014_processing
import generate_dpe_annexes_scripts.td001_merge
import generate_dpe_annexes_scripts.text_matching_dict
import generate_dpe_annexes_scripts.utils

In [ ]:
reload_package(main)
reload_package(generate_dpe_annexes_scripts.conversion_normalisation)

reload_package(generate_dpe_annexes_scripts.td011_td012_processing)
reload_package(generate_dpe_annexes_scripts.td013_td014_processing)
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
reload_package(generate_dpe_annexes_scripts.td001_merge)
reload_package(generate_dpe_annexes_scripts.utils)

reload_package(main)
reload_package(generate_dpe_annexes_scripts.conversion_normalisation)

reload_package(generate_dpe_annexes_scripts.td011_td012_processing)
reload_package(generate_dpe_annexes_scripts.td013_td014_processing)
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
reload_package(generate_dpe_annexes_scripts.td001_merge)

In [ ]:
from generate_dpe_annexes_scripts.text_matching_dict import *

In [ ]:
from main import *

In [ ]:
td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)
td013 = pd.read_csv(dept_dir / 'td013_installation_ecs.csv', dtype=str)
td014 = pd.read_csv(dept_dir / 'td014_generateur_ecs.csv', dtype=str)

td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg = run_system_processing(td001, td006,
                                                                                                td011, td012,
                                                                                                td013, td014)
round_float_cols(td001_sys_ch_agg).to_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv')
round_float_cols(td001_sys_ecs_agg).to_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv')
round_float_cols(td011_p).to_csv(annexe_dept_dir / 'td011_installation_chauffage_annexe.csv')
round_float_cols(td012_p).to_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv')
round_float_cols(td013_p).to_csv(annexe_dept_dir / 'td013_installation_ecs_annexe.csv')
round_float_cols(td014_p).to_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv')

In [ ]:
td001, td006, td011, td012, td013, td014 = merge_td001_dpe_id_system(td001, td006, td011, td012, td013, td014)


table = merge_without_duplicate_columns(td012,td012_p,on='td012_generateur_chauffage_id')


In [ ]:
td011 = merge_without_duplicate_columns(td011,td011_p,on='td011_installation_chauffage_id')


In [ ]:
table = merge_without_duplicate_columns(td012,td012_p,on='td012_generateur_chauffage_id')


In [ ]:
table=table.astype(td012_types)

In [ ]:
td011['type_installation_ch'] = td011.tv025_type_installation.replace(
    type_installation_conv_dict['tv025_type_installation'])
#table = td012.copy()
table['type_energie_ch'] = table["tv045_energie"].astype('string').fillna('indetermine').replace(
    ener_conv_dict['tv045_energie'])
table['tr004_description'] = table["tr004_description"].astype('string').fillna('indetermine').replace(
    ener_conv_dict['tr004_description'])

is_rpn = table.rpn > 0
is_rpint = table.rpint > 0
is_chaudiere = is_rpint | is_rpn
is_chaudiere = is_chaudiere | ~table.tv038_puissance_nominale_id.isnull()
# all text description raw concat
gen_ch_concat_txt_desc = table['tv031_type_generateur'].astype('string').replace(np.nan, '') + ' '
gen_ch_concat_txt_desc.loc[is_chaudiere] += 'chaudiere '
gen_ch_concat_txt_desc += table['tv036_type_chaudiere'].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table["tv030_type_installation"].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table["tv032_type_generateur"].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table['tv035_type_chaudiere'].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table['tv036_type_generation'].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table["tv030_type_installation"].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table["tr004_description"].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table["type_energie_ch"].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table['tv046_nom_reseau'].isnull().replace({False: 'réseau de chaleur',
                                                                      True: ""})
gen_ch_concat_txt_desc = gen_ch_concat_txt_desc.str.lower().apply(lambda x: strip_accents(x))

table['gen_ch_concat_txt_desc'] = gen_ch_concat_txt_desc

table['gen_ch_concat_txt_desc'] = table['gen_ch_concat_txt_desc'].apply(lambda x: clean_str(x))

# calcul gen_ch_lib_infer par matching score text.
unique_gen_ch = table.gen_ch_concat_txt_desc.unique()
gen_ch_lib_infer_dict = {k: affect_lib_by_matching_score(k, td012_gen_ch_search_dict) for k in
                         unique_gen_ch}
table['gen_ch_lib_infer'] = table.gen_ch_concat_txt_desc.replace(gen_ch_lib_infer_dict)

# recup/fix PAC
is_pac = (table.coefficient_performance > 2) | (table.rendement_generation > 2)
table.loc[is_pac, 'gen_ch_lib_infer'] = table.loc[is_pac, 'coefficient_performance'].replace(pac_dict)
is_ind = is_pac & (~table.loc[is_pac, 'gen_ch_lib_infer'].isin(pac_dict.values()))
table.loc[is_ind, 'gen_ch_lib_infer'] = table.loc[is_ind, 'rendement_generation'].replace(pac_dict)
is_ind = is_pac & (~table.loc[is_pac, 'gen_ch_lib_infer'].isin(pac_dict.values()))
table.loc[is_ind, 'gen_ch_lib_infer'] = 'pac indetermine'

# recup poele bois from chauffage indetermine
is_bois_ind = table.gen_ch_lib_infer == 'chauffage bois indetermine'

table.loc[is_bois_ind, 'gen_ch_lib_infer'] = table.loc[is_bois_ind, 'rendement_generation'].replace(poele_dict)

# recup poele fioul from chauffage indetermine
is_fioul_ind = table.gen_ch_lib_infer == 'chauffage fioul indetermine'

table.loc[is_fioul_ind, 'gen_ch_lib_infer'] = table.loc[is_fioul_ind, 'rendement_generation'].replace(poele_dict)

# recup reseau chaleur
non_aff = table.gen_ch_lib_infer == 'indetermine'

reseau_infer = non_aff & (table.rendement_generation == 0.97) & (table.tr004_description == 'Autres énergies')

table.loc[reseau_infer, 'gen_ch_lib_infer'] = 'reseau de chaleur'
table.loc[reseau_infer, 'type_energie_ch'] = 'reseau de chaleur'

table['gen_ch_lib_infer_simp'] = table.gen_ch_lib_infer.replace(gen_ch_lib_simp_dict)

# fix chaudiere elec

bool_cei = table.gen_ch_lib_infer == 'chauffage electrique indetermine'
bool_ce = table.rendement_generation == 0.77

table.loc[(bool_cei) & (bool_ce), 'gen_ch_lib_infer'] = 'chaudiere electrique'

rendement_gen_u = table[['rendement_generation', 'coefficient_performance']].max(axis=1)

table['rendement_generateur_u']=rendement_gen_u

# chauffage effet joule

s_rendement = pd.Series(index=table.index)
s_rendement[:] = 1
for rendement in ['rendement_distribution_systeme_chauffage',
                  'rendement_emission_systeme_chauffage']:
    r = table[rendement].astype(float)
    r[r == 0] = 1
    r[r.isnull()] = 1
    s_rendement = s_rendement * r

rendement_gen_u[rendement_gen_u == 0] = 1
rendement_gen_u[rendement_gen_u.isnull()] = 1
s_rendement = s_rendement * rendement_gen_u
table['besoin_ch_infer'] = table['consommation_chauffage'] * s_rendement

In [ ]:
table.gen_ch_lib_infer.fillna('NAAAAAAAAAAAAAAAA').value_counts()

In [ ]:
td001_sys_ch.type_installation_ch_concat=td001_sys_ch.type_installation_ch_concat.str.replace('Chauffage Individuel',
                                                                                              'individuel').str.replace('Chauffage Collectif',
                                                                                                                        'collectif')
td001 =td001.rename(columns={'id':'td001_dpe_id'})

In [ ]:
    gen_ch_concat_txt_desc = table['tv031_type_generateur'].astype('string').replace(np.nan, '') + ' '
    gen_ch_concat_txt_desc.loc[is_chaudiere] += 'chaudiere '
    gen_ch_concat_txt_desc += table['tv036_type_chaudiere'].astype('string').replace(np.nan, ' ') + ' '
    gen_ch_concat_txt_desc += table["tv030_type_installation"].astype('string').replace(np.nan, ' ') + ' '
    gen_ch_concat_txt_desc += table["tv032_type_generateur"].astype('string').replace(np.nan, ' ') + ' '
    gen_ch_concat_txt_desc += table['tv035_type_chaudiere'].astype('string').replace(np.nan, ' ') + ' '
    gen_ch_concat_txt_desc += table['tv036_type_generation'].astype('string').replace(np.nan, ' ') + ' '
    gen_ch_concat_txt_desc += table["tv030_type_installation"].astype('string').replace(np.nan, ' ') + ' '
    gen_ch_concat_txt_desc += table["tr004_description"].astype('string').replace(np.nan, ' ') + ' '
    gen_ch_concat_txt_desc += table["type_energie_ch"].astype('string').replace(np.nan, ' ') + ' '
    gen_ch_concat_txt_desc += table['tv046_nom_reseau'].isnull().replace({False: 'réseau de chaleur',
                                                                          True: ""})
    gen_ch_concat_txt_desc = gen_ch_concat_txt_desc.str.lower().apply(lambda x: strip_accents(x))

    table['gen_ch_concat_txt_desc'] = gen_ch_concat_txt_desc

    table['gen_ch_concat_txt_desc'] = table['gen_ch_concat_txt_desc'].apply(lambda x: clean_str(x))

In [ ]:
    unique_gen_ch = table.gen_ch_concat_txt_desc.unique()
    gen_ch_lib_infer_dict = {k: affect_lib_by_matching_score(k, td012_gen_ch_search_dict) for k in
                             unique_gen_ch}
    table['gen_ch_lib_infer'] = table.gen_ch_concat_txt_desc.replace(gen_ch_lib_infer_dict)

In [ ]:
table.query('gen_ch_lib_infer=="indetermine"').columns.tolist()

In [ ]:
table.query('gen_ch_lib_infer=="indetermine"')[['tr004_type_energie_id','rendement_generation',
                                                'rendement_distribution_systeme_chauffage',
                                              'gen_ch_concat_txt_desc']].rendement_distribution_systeme_chauffage.value_counts()

In [ ]:
table.query('gen_ch_lib_infer=="indetermine"')[['type_energie_ch','tr004_description','tv045_energie']]

In [ ]:
td012_p.gen_ch_lib_infer.value_counts()

In [ ]:
    td001, td006, td011, td012, td013, td014 = merge_td001_dpe_id_system(td001, td006, td011, td012, td013, td014)


table = merge_without_duplicate_columns(td012,td012_p,on='td012_generateur_chauffage_id')


In [ ]:
table.query('gen_ch_lib_infer=="chauffage electrique indetermine"').to_dict(orient='records')

# td002/td016 processing

In [ ]:
from generate_dpe_annexes_scripts.td002_td016_processing import merge_td002_td016_trtrv,extract_type_energie_from_td002_td016

In [ ]:
td002_p,td016_p=merge_td002_td016_trtrv(td002,td016)
td001_ener_from_facture = extract_type_energie_from_td002_td016(td001,td002_p,td016_p)

In [ ]:
td001_ener_from_facture

# load dicts

In [738]:
from generate_dpe_annexes_scripts.utils import reload_package

In [1210]:
import generate_dpe_annexes_scripts.text_matching_dict 
import os
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
from generate_dpe_annexes_scripts.text_matching_dict import *

from generate_dpe_annexes_scripts.conversion_normalisation import *
reload_package(generate_dpe_annexes_scripts.conversion_normalisation)
from generate_dpe_annexes_scripts.conversion_normalisation import *
reload_package(generate_dpe_annexes_scripts.td003_td005_text_extraction_processing)

from generate_dpe_annexes_scripts.td003_td005_text_extraction_processing import extract_td005_ch_variables,extract_td003_ch_variables

from generate_dpe_annexes_scripts.td003_td005_text_extraction_processing import extract_td005_ecs_variables,extract_td003_ecs_variables

In [1211]:
gen_ch_search_dict_flat.keys() -set(td012_gen_ch_search_dict.keys())

{'chaudiere bois exact',
 'chaudiere electrique exact',
 'chaudiere energie indetermine basse temperature',
 'chaudiere energie indetermine condensation',
 'chaudiere energie indetermine indetermine',
 'chaudiere energie indetermine standard',
 'convecteurs electriques',
 'pac air/eau en releve de chaudiere',
 'pac eau/eau en releve de chaudiere',
 'pac geothermique en releve de chaudiere',
 'pac indetermine',
 'pac indetermine en releve de chaudiere',
 'panneaux rayonnants electriques',
 'poele ou insert indetermine',
 'radiateurs gaz'}

In [741]:
set(td014_gen_ecs_search_dict.keys())-gen_ecs_search_dict_flat.keys()

set()

In [707]:
energie_search_dict

{'gaz': ['gaz'],
 'electricite': [('elec*', 'elec.', 'joule*')],
 'reseau de chaleur': [('"reseau de chaleur"',
   '"reseaux de chaleurs"',
   '"reseaux de chaleur"',
   '"reseau chaleur"',
   'urbain',
   'rcu',
   '(raccordement AND reseau)')],
 'fioul': [('fioul', 'mazout', 'fuel')],
 'bois': [('bois',
   'bois//biomasse',
   'bio*',
   'flamme AND verte',
   'granule*',
   'pellet*',
   'buche*')],
 'gpl/butane/propane': [('propane', 'butane', 'gpl')],
 'charbon': ['charbon']}

In [701]:
gen_ecs_search_dict_flat.keys() -set(td014_gen_ecs_search_dict.keys())

{'abscence ecs solaire',
 'chaudiere bois exact',
 'chaudiere electrique exact',
 'chaudiere energie indetermine basse temperature',
 'chaudiere energie indetermine condensation',
 'chaudiere energie indetermine indetermine',
 'chaudiere energie indetermine standard',
 'chauffe-eau independant indetermine',
 'ecs bois indetermine',
 'ecs solaire',
 'poele bouilleur bois',
 'production mixte fioul',
 'production mixte gaz',
 'production mixte indetermine'}

# search Chauffage

In [742]:
from generate_dpe_annexes_scripts import text_matching_dict
reload_package(text_matching_dict)
from generate_dpe_annexes_scripts.text_matching_dict import solaire_ch_search_dict

In [743]:
gen_ch_lib_ft, type_installation_ch_ft, energie_ch_ft, solaire_ch_ft = extract_td005_ch_variables(td005)


gen_ch_lib_desc, type_installation_ch_desc, energie_ch_desc, solaire_ch_desc = extract_td003_ch_variables(td003)


D:\Cstbdata\MONWORKSPACE\Gitlab\de-dpe-processing\data_processing_cstb\traitement_donnees_metier\generate_dpe_annexes_scripts\td003_td005_text_extraction_processing.py:23: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  td005_ch.descriptif = td005_ch.valeur_renseignee.fillna('indetermine')
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# search ecs

In [744]:
gen_ecs_lib_ft, type_installation_ecs_ft, energie_ecs_ft = extract_td005_ecs_variables(td005)


gen_ecs_lib_desc, type_installation_ecs_desc, energie_ecs_desc = extract_td003_ecs_variables(td003)


# fusion

## méthodologie de fusion

TBD

In [1003]:
td001_sys=td001.rename(columns ={'id':'td001_dpe_id'})[['td001_dpe_id','tr002_type_batiment_id']].merge(td001_sys_ch[['td001_dpe_id',
                                                                                             'gen_ch_lib_infer_concat','mix_energetique_ch'
                                                                                            ]],on='td001_dpe_id',
                                                                              how='left')
td001_sys=td001_sys.merge(td001_sys_ecs[['td001_dpe_id','gen_ecs_lib_infer_concat']],on='td001_dpe_id',
                                                                              how='left')

## Chauffage


### préparation de la donnée

#### generateur chauffage

In [1212]:
# libéllés des generateurs chauffage issues des données textes
gen_ch_lib_from_txt=pd.concat([gen_ch_lib_desc[['label','category','td001_dpe_id']],
                               gen_ch_lib_ft[['label','category','td001_dpe_id']]],axis=0)

gen_ch_lib_from_txt =gen_ch_lib_from_txt.sort_values(by=['td001_dpe_id','category','label'])

gen_ch_lib_from_txt = gen_ch_lib_from_txt.drop_duplicates(subset=['category','td001_dpe_id'],keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
gen_ch_lib_from_txt.label = gen_ch_lib_from_txt.label.str.replace(' exact','')

gen_ch_lib_from_txt['source']='txt'

# libéllés des générateurs chauffage issus de la table td012
gen_ch_lib_from_data = td012_p[['td001_dpe_id','gen_ch_lib_infer']].copy()

gen_ch_lib_from_data.columns=['td001_dpe_id','label']

gen_ch_lib_from_data['category']=gen_ch_lib_from_data.label.replace(reverse_cat_gen_ch)

gen_ch_lib_from_data['label']=pd.Categorical(gen_ch_lib_from_data.label,categories=list(gen_ch_search_dict_flat.keys())+['indetermine'],ordered=True)
gen_ch_lib_from_data['source']='data'

tr003_desc = td011_p.loc[td011_p.tr003_description.isin(tr003_desc_to_gen.keys())]

tr003_desc =tr003_desc[['td001_dpe_id','tr003_description']]
tr003_desc.columns=['td001_dpe_id','label']
tr003_desc.label=tr003_desc.label.replace(tr003_desc_to_gen)
tr003_desc['category']=tr003_desc.label.replace(reverse_cat_gen_ch)

tr003_desc['label']=pd.Categorical(tr003_desc.label,categories=list(gen_ch_search_dict_flat.keys())+['indetermine'],ordered=True)
tr003_desc['source']='data'

# concat. 
gen_ch_lib = pd.concat([gen_ch_lib_from_data,tr003_desc,gen_ch_lib_from_txt],axis=0)

gen_ch_lib['source']=pd.Categorical(gen_ch_lib.source,categories=['data','txt'],ordered=True)
gen_ch_lib['category']=pd.Categorical(gen_ch_lib.category,categories=list(gen_ch_search_dict.keys()),ordered=True)



In [1213]:
gen_ch_lib.label.value_counts()

chauffage electrique indetermine                   92059
chauffage gaz indetermine                          62734
chaudiere gaz indetermine                          29168
poele ou insert bois                               27045
radiateurs electriques                             25994
chaudiere gaz condensation                         25498
panneaux rayonnants electriques nfc                21462
convecteurs electriques nfc                        20704
chauffage bois indetermine                         19074
chauffage fioul indetermine                        14396
reseau de chaleur                                  14357
chaudiere gaz standard                             13933
chaudiere fioul indetermine                        12005
poele ou insert indetermine                         9110
plafonds/planchers rayonnants electriques nfc       8239
chaudiere gaz basse temperature                     7527
chaudiere electrique                                7310
pac air/air                    

In [1196]:
tr003_desc.label.value_counts()

poele ou insert indetermine               7303
pac indetermine en releve de chaudiere     289
convecteurs bi-jonction electriques         95
chaudiere bois                              47
Name: label, dtype: int64

#### type installation

In [1005]:


# libéllés des generateurs chauffage issues des données textes
type_installation_ch_from_txt = pd.concat([type_installation_ch_desc[['label','td001_dpe_id']],
                               type_installation_ch_ft[['label','td001_dpe_id']]],axis=0)

type_installation_ch_from_txt['label'] = pd.Categorical(type_installation_ch_from_txt['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)



type_installation_ch_from_txt = type_installation_ch_from_txt.sort_values(by=['td001_dpe_id','label'])

type_installation_ch_from_txt = type_installation_ch_from_txt.drop_duplicates(subset=['td001_dpe_id'],keep='first')

type_installation_ch_from_txt['source'] = 'txt'

type_installation_ch_from_data = td011_p[['td001_dpe_id','type_installation_ch']].copy()
type_installation_ch_from_data.columns = ['td001_dpe_id','label']
type_installation_ch_from_data['label'] = type_installation_ch_from_data['label'].fillna('indetermine')
type_installation_ch_from_data['label'] = pd.Categorical(type_installation_ch_from_data['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)
type_installation_ch_from_data['source'] = 'data'
type_installation_ch = pd.concat([type_installation_ch_from_data,type_installation_ch_from_txt],axis=0)


type_installation_ch['is_indetermine']=type_installation_ch.label=='indetermine'

#### energie

In [1006]:


# libéllés des generateurs chauffage issues des données textes
energie_ch_from_txt = pd.concat([energie_ch_desc[['label','td001_dpe_id']],
                               energie_ch_ft[['label','td001_dpe_id']]],axis=0)

energie_ch_from_txt['label'] = pd.Categorical(energie_ch_from_txt['label'],
                                                    categories=energie_normalise_ordered,ordered=True)



energie_ch_from_txt = energie_ch_from_txt.sort_values(by=['td001_dpe_id','label'])


energie_ch_from_txt['source'] = 'txt'

energie_ch_from_data = td012_p[['td001_dpe_id','type_energie_ch']].copy()
energie_ch_from_data.columns = ['td001_dpe_id','label']
energie_ch_from_data['label'] = energie_ch_from_data['label'].fillna('indetermine')
energie_ch_from_data['label'] = pd.Categorical(energie_ch_from_data['label'],
                                                    categories=energie_normalise_ordered,ordered=True)
energie_ch_from_data['source'] = 'data'
energie_ch = pd.concat([energie_ch_from_data,energie_ch_from_txt],axis=0)


energie_ch['is_indetermine']=energie_ch.label=='indetermine'

### amélioration des labels chaudières

Problème : dans les données ou le texte on manque parfois de l'information type energie et/ou type de chaudiere (condensation/BT etc..) on cherche le maximum d'information des diverses sources pour recréer le bon libéllé

In [1007]:
is_chaudiere_ener_ind = gen_ch_lib.label.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind = is_chaudiere_ener_ind &(~gen_ch_lib.label.str.contains('chaudiere energie indetermine indetermine'))
is_chaudiere_ind = gen_ch_lib.label.str.contains('chaudiere')&gen_ch_lib.label.str.contains('indetermine')&(~is_chaudiere_ener_ind)


gen_ch_lib['chaudiere_ener_ind']=is_chaudiere_ener_ind
gen_ch_lib['chaudiere_ind']=is_chaudiere_ind

sum_inds=gen_ch_lib.groupby('td001_dpe_id')[['chaudiere_ener_ind','chaudiere_ind']].sum()

id_sel=sum_inds.loc[(sum_inds>0).min(axis=1)].index

ind_sel = gen_ch_lib.td001_dpe_id.isin(id_sel)&gen_ch_lib.chaudiere_ind
chaudiere_ind=gen_ch_lib.loc[ind_sel]

chaudiere_ener_ind=gen_ch_lib.loc[gen_ch_lib.td001_dpe_id.isin(id_sel)&gen_ch_lib.chaudiere_ener_ind]

suffix=chaudiere_ener_ind.set_index('td001_dpe_id').label.apply(lambda x:x.split('energie indetermine')[-1].strip())
suffix.name='suffix'

chaudiere_ind = chaudiere_ind.merge(suffix,on='td001_dpe_id',how='left')

chaudiere_ind.label =chaudiere_ind.label.astype(str)+ ' '+chaudiere_ind.suffix

chaudiere_ind.label=chaudiere_ind.label.str.replace('indetermine ','').str.strip()

gen_ch_lib.loc[ind_sel,'label']=chaudiere_ind.label.values

### amelioration des labels de pac

### drop des labels chauffage par défaut lorsqu'il existe mieux. 

#### elec

In [1010]:
# on identifie les dpe pour lesquels on a des champs label elec determine (pac chaudiere ou effet joule) et le champs indetermine
elec_ind = (gen_ch_lib.set_index('td001_dpe_id').label == 'chauffage electrique indetermine')
elec_deter= gen_ch_lib.set_index('td001_dpe_id').label.str.contains('electrique|pac')&(~elec_ind)
elec_deter = elec_deter.groupby('td001_dpe_id').max()
elec_ind = elec_ind.groupby('td001_dpe_id').max()

sel = elec_ind&elec_deter
sel = sel[sel]
sel = sel.to_frame('label').reset_index()
sel['label']='chauffage electrique indetermine'
sel['to_drop']=True

gen_ch_lib = gen_ch_lib.merge(sel,on=['td001_dpe_id','label'],how='left')

gen_ch_lib = gen_ch_lib.loc[gen_ch_lib.to_drop!=True]
del gen_ch_lib['to_drop']

#### energie à combustion

In [1011]:
# pour les autres energies les label contiennent l'energie de chauffage
for ener in energie_combustion_mods:
    ener_ind = (gen_ch_lib.set_index('td001_dpe_id').label == f'chauffage {ener} indetermine')
    ener_deter= gen_ch_lib.set_index('td001_dpe_id').label.str.contains(f'{ener}')&(~ener_ind)
    ener_deter = ener_deter.groupby('td001_dpe_id').max()
    ener_ind = ener_ind.groupby('td001_dpe_id').max()

    sel = ener_ind&ener_deter
    sel = sel[sel]
    sel = sel.to_frame('label').reset_index()
    sel['label']=f'chauffage {ener} indetermine'
    sel['to_drop']=True
    gen_ch_lib = gen_ch_lib.merge(sel,on=['td001_dpe_id','label'],how='left')

    gen_ch_lib = gen_ch_lib.loc[gen_ch_lib.to_drop!=True]
    del gen_ch_lib['to_drop']

### fusion

#### generateur chauffage

In [1012]:
gen_ch_lib = gen_ch_lib.sort_values(by=['td001_dpe_id','category','label','source'])


gen_ch_lib = gen_ch_lib.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ch_lib = gen_ch_lib.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ch_lib=gen_ch_lib.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ch_lib = gen_ch_lib.to_frame('gen_ch_lib_final')

td001_sys = td001_sys.merge(gen_ch_lib.reset_index(),on='td001_dpe_id',how='left')


#### chauffage solaire

In [1013]:
solaire_ch_txt=pd.concat([solaire_ch_ft[['label','td001_dpe_id']],solaire_ch_desc[['label','td001_dpe_id']]],axis=0)

solaire_ch_txt = solaire_ch_txt.sort_values('label').drop_duplicates(subset=['td001_dpe_id'])

solaire_ch_txt= solaire_ch_txt.loc[solaire_ch_txt.label=='solaire']

is_ch_solaire_txt=solaire_ch_txt.groupby('td001_dpe_id').label.count().astype(float)>0

td011_p['tr003_solaire'] = td011_p['tr003_solaire'].astype(float)

is_ch_solaire_data = td011_p.groupby('td001_dpe_id').tr003_solaire.max()>0

td001_sys = td001_sys.merge(is_ch_solaire_data.to_frame('is_ch_solaire_data'),on='td001_dpe_id',how='left')

td001_sys = td001_sys.merge(is_ch_solaire_txt.to_frame('is_ch_solaire_txt'),on='td001_dpe_id',how='left')

td001_sys['is_ch_solaire'] = td001_sys.is_ch_solaire_txt.fillna(0).astype(float)+td001_sys.is_ch_solaire_data.fillna(0).astype(float)
td001_sys['is_ch_solaire'] = td001_sys['is_ch_solaire'] > 0

is_ch_solaire = td001_sys.is_ch_solaire==True

td001_sys.loc[is_ch_solaire,'gen_ch_lib_final']='chauffage solaire + '+td001_sys.loc[is_ch_solaire,'gen_ch_lib_final']

#### type installation

In [1014]:
type_installation_ch = type_installation_ch.sort_values(by=['td001_dpe_id','is_indetermine',
                                     'source','label'])
type_installation_ch = type_installation_ch.drop_duplicates(subset=['td001_dpe_id'])

type_installation_ch = type_installation_ch.set_index('td001_dpe_id').label.to_frame('type_installation_ch')

td001_sys = td001_sys.merge(type_installation_ch.reset_index(),on='td001_dpe_id',how='left')


#### energie

In [1015]:

energie_ch = energie_ch.sort_values(by=['td001_dpe_id','label','source'])

energie_ch = energie_ch.drop_duplicates(subset=['td001_dpe_id','label'])

energie_ch=energie_ch.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x))).to_frame('type_energie_ch')




td001_sys = td001_sys.merge(energie_ch.reset_index(),on='td001_dpe_id',how='left')


#  ecs

### preparation de la donnée


#### generateur ecs

In [1016]:


# libéllés des generateurs chauffage issues des données textes
gen_ecs_lib_from_txt=pd.concat([gen_ecs_lib_desc[['label','category','td001_dpe_id']],
                               gen_ecs_lib_ft[['label','category','td001_dpe_id']]],axis=0)

gen_ecs_lib_from_txt =gen_ecs_lib_from_txt.sort_values(by=['td001_dpe_id','category','label'])

gen_ecs_lib_from_txt = gen_ecs_lib_from_txt.drop_duplicates(subset=['category','td001_dpe_id'],keep='first')

# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
gen_ecs_lib_from_txt.label = gen_ecs_lib_from_txt.label.str.replace(' exact','')

gen_ecs_lib_from_txt['source']='txt'

# libéllés des générateurs chauffage issus de la table td012
gen_ecs_lib_from_data = td014_p[['td001_dpe_id','gen_ecs_lib_infer']].copy()

gen_ecs_lib_from_data.columns=['td001_dpe_id','label']
gen_ecs_lib_from_data['label']=gen_ecs_lib_from_data['label'].fillna('indetermine')
gen_ecs_lib_from_data['category']=gen_ecs_lib_from_data.label.replace(reverse_cat_gen_ecs)

gen_ecs_lib_from_data['label']=pd.Categorical(gen_ecs_lib_from_data.label,categories=list(gen_ecs_search_dict_flat.keys())+['indetermine'],ordered=True)
gen_ecs_lib_from_data['label']=gen_ecs_lib_from_data['label'].fillna('indetermine')

gen_ecs_lib_from_data['source']='data'

# concat. 
gen_ecs_lib = pd.concat([gen_ecs_lib_from_data,gen_ecs_lib_from_txt],axis=0)

gen_ecs_lib['source']=pd.Categorical(gen_ecs_lib.source,categories=['data','txt'],ordered=True)
gen_ecs_lib['category']=pd.Categorical(gen_ecs_lib.category,categories=list(gen_ecs_search_dict.keys()),ordered=True)



#### type installation

In [1017]:


# libéllés des generateurs chauffage issues des données textes
type_installation_ecs_from_txt = pd.concat([type_installation_ecs_desc[['label','td001_dpe_id']],
                               type_installation_ecs_ft[['label','td001_dpe_id']]],axis=0)

type_installation_ecs_from_txt['label'] = pd.Categorical(type_installation_ecs_from_txt['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)



type_installation_ecs_from_txt = type_installation_ecs_from_txt.sort_values(by=['td001_dpe_id','label'])

type_installation_ecs_from_txt = type_installation_ecs_from_txt.drop_duplicates(subset=['td001_dpe_id'],keep='first')

type_installation_ecs_from_txt['source'] = 'txt'

type_installation_ecs_from_data = td014_p[['td001_dpe_id','type_installation_ecs']].copy()
type_installation_ecs_from_data.columns = ['td001_dpe_id','label']
type_installation_ecs_from_data['label'] = type_installation_ecs_from_data['label'].fillna('indetermine')
type_installation_ecs_from_data['label'] = pd.Categorical(type_installation_ecs_from_data['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)
type_installation_ecs_from_data['source'] = 'data'
type_installation_ecs = pd.concat([type_installation_ecs_from_data,type_installation_ecs_from_txt],axis=0)


type_installation_ecs['is_indetermine']=type_installation_ecs.label=='indetermine'

#### energie 

In [1018]:


# libéllés des generateurs chauffage issues des données textes
energie_ecs_from_txt = pd.concat([energie_ecs_desc[['label','td001_dpe_id']],
                               energie_ecs_ft[['label','td001_dpe_id']]],axis=0)

energie_ecs_from_txt['label'] = pd.Categorical(energie_ecs_from_txt['label'],
                                                    categories=energie_normalise_ordered,ordered=True)



energie_ecs_from_txt = energie_ecs_from_txt.sort_values(by=['td001_dpe_id','label'])

energie_ecs_from_txt['source'] = 'txt'

energie_ecs_from_data = td014_p[['td001_dpe_id','type_energie_ecs']].copy()
energie_ecs_from_data.columns = ['td001_dpe_id','label']
energie_ecs_from_data['label'] = energie_ecs_from_data['label'].fillna('indetermine')
energie_ecs_from_data['label'] = pd.Categorical(energie_ecs_from_data['label'],
                                                    categories=energie_normalise_ordered,ordered=True)
energie_ecs_from_data['source'] = 'data'
energie_ecs = pd.concat([energie_ecs_from_data,energie_ecs_from_txt],axis=0)


energie_ecs['is_indetermine']=energie_ecs.label=='indetermine'

### amélioration des labels chaudières

Problème : dans les données ou le texte on manque parfois de l'information type energie et/ou type de chaudiere (condensation/BT etc..) on cherche le maximum d'information des diverses sources pour recréer le bon libéllé

In [1019]:
is_chaudiere_ener_ind = gen_ecs_lib.label.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind = is_chaudiere_ener_ind &(~gen_ecs_lib.label.str.contains('chaudiere energie indetermine indetermine'))
is_chaudiere_ind = gen_ecs_lib.label.str.contains('chaudiere')&gen_ecs_lib.label.str.contains('indetermine')&(~is_chaudiere_ener_ind)


gen_ecs_lib['chaudiere_ener_ind']=is_chaudiere_ener_ind
gen_ecs_lib['chaudiere_ind']=is_chaudiere_ind

sum_inds=gen_ecs_lib.groupby('td001_dpe_id')[['chaudiere_ener_ind','chaudiere_ind']].sum()

id_sel=sum_inds.loc[(sum_inds>0).min(axis=1)].index

ind_sel = gen_ecs_lib.td001_dpe_id.isin(id_sel)&gen_ecs_lib.chaudiere_ind
chaudiere_ind=gen_ecs_lib.loc[ind_sel]

chaudiere_ener_ind=gen_ecs_lib.loc[gen_ecs_lib.td001_dpe_id.isin(id_sel)&gen_ecs_lib.chaudiere_ener_ind]

suffix=chaudiere_ener_ind.set_index('td001_dpe_id').label.apply(lambda x:x.split('energie indetermine')[-1].strip())
suffix.name='suffix'

chaudiere_ind = chaudiere_ind.merge(suffix,on='td001_dpe_id',how='left')

chaudiere_ind.label =chaudiere_ind.label.astype(str)+ ' '+chaudiere_ind.suffix

chaudiere_ind.label=chaudiere_ind.label.str.replace('indetermine ','').str.strip()

gen_ecs_lib.loc[ind_sel,'label']=chaudiere_ind.label.values

### drop des labels chauffage par défaut lorsqu'il existe mieux. 

#### elec

In [1020]:
# on identifie les dpe pour lesquels on a des champs label elec determine (pac chaudiere ou effet joule) et le champs indetermine
elec_ind = (gen_ecs_lib.set_index('td001_dpe_id').label == 'ecs electrique indetermine')
elec_deter= gen_ecs_lib.set_index('td001_dpe_id').label.str.contains('electrique|thermodynamique')&(~elec_ind)
elec_deter = elec_deter.groupby('td001_dpe_id').max()
elec_ind = elec_ind.groupby('td001_dpe_id').max()

sel = elec_ind&elec_deter
sel = sel[sel]
sel = sel.to_frame('label').reset_index()
sel['label']='ecs electrique indetermine'
sel['to_drop']=True

gen_ecs_lib = gen_ecs_lib.merge(sel,on=['td001_dpe_id','label'],how='left')

gen_ecs_lib = gen_ecs_lib.loc[gen_ecs_lib.to_drop!=True]
del gen_ecs_lib['to_drop']

#### energie à combustion

In [1021]:
# pour les autres energies les label contiennent l'energie de chauffage
for ener in energie_combustion_mods:
    ener_ind = (gen_ecs_lib.set_index('td001_dpe_id').label == f'ecs {ener} indetermine')
    ener_deter= gen_ecs_lib.set_index('td001_dpe_id').label.str.contains(f'{ener}')&(~ener_ind)
    ener_deter = ener_deter.groupby('td001_dpe_id').max()
    ener_ind = ener_ind.groupby('td001_dpe_id').max()

    sel = ener_ind&ener_deter
    sel = sel[sel]
    sel = sel.to_frame('label').reset_index()
    sel['label']=f'ecs {ener} indetermine'
    sel['to_drop']=True
    gen_ecs_lib = gen_ecs_lib.merge(sel,on=['td001_dpe_id','label'],how='left')

    gen_ecs_lib = gen_ecs_lib.loc[gen_ecs_lib.to_drop!=True]
    del gen_ecs_lib['to_drop']

### fusion

In [1022]:
td001_sys = td001_sys.loc[:,~td001_sys.columns.str.contains('_x|_y') ]

In [1023]:
priorisation_ecs={'solaire':"solaire",
 'abscence solaire':"solaire",
 'ecs thermodynamique':"principal",
 'chaudiere bois':"principal",
 'chaudiere':"principal",
 'production mixte indetermine':"secondaire",
 'reseau de chaleur':"principal",
 'effet joule':"secondaire",
 'chauffe-eau_independant':"defaut",
 'ecs electrique indetermine':"secondaire",
 'ecs bois indetermine':"defaut",
 'ecs fioul indetermine':"defaut",
 'ecs gaz indetermine':"defaut",
 'ecs gpl/butane/propane indetermine':"defaut",
 'ecs charbon indetermine':"defaut",}

In [1024]:
gen_ecs_lib['priorite']=gen_ecs_lib.category.replace(priorisation_ecs)

In [1136]:
gen_ecs_lib[]

,td001_dpe_id,label,category,source,chaudiere_ener_ind,chaudiere_ind,priorite
0,291,ballon a accumulation electrique,effet joule,data,False,False,secondaire
1,493,ballon a accumulation electrique,effet joule,data,False,False,secondaire
2,708,ballon a accumulation electrique,effet joule,data,False,False,secondaire
3,1466,chaudiere fioul indetermine,chaudiere,data,False,True,principal
4,1708,ballon a accumulation electrique,effet joule,data,False,False,secondaire
...,...,...,...,...,...,...,...
434915,99972,ballon a accumulation electrique,effet joule,txt,False,False,secondaire
434916,999768,chauffe-eau independant indetermine,chauffe-eau_independant,txt,False,False,defaut
434917,999768,ballon a accumulation electrique,effet joule,txt,False,False,secondaire
434918,999890,chauffe-eau independant indetermine,chauffe-eau_independant,txt,False,False,defaut


#### traitement des generateurs d'ECS prioritaires

In [1140]:
gen_ecs_lib_princ = gen_ecs_lib.query('priorite=="principal"')

In [1026]:


gen_ecs_lib_princ = gen_ecs_lib_princ.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ecs_lib_princ = gen_ecs_lib_princ.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ecs_lib_princ=gen_ecs_lib_princ.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ecs_lib_princ = gen_ecs_lib_princ.to_frame('gen_ecs_lib_final')



td001_sys = td001_sys.merge(gen_ecs_lib_princ.reset_index(),on='td001_dpe_id',how='left')


#### traitement des generateurs d'ecs secondaires

In [1027]:
gen_ecs_lib_sec = gen_ecs_lib.query('priorite=="secondaire"')

gen_ecs_lib_sec = gen_ecs_lib_sec.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ecs_lib_sec = gen_ecs_lib_sec.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ecs_lib_sec=gen_ecs_lib_sec.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ecs_lib_sec = gen_ecs_lib_sec.to_frame('gen_ecs_lib_final_sec')

td001_sys = td001_sys.merge(gen_ecs_lib_sec.reset_index(),on='td001_dpe_id',how='left')

null=td001_sys.gen_ecs_lib_final.isnull()

td001_sys.loc[null,'gen_ecs_lib_final']=td001_sys.loc[null,'gen_ecs_lib_final_sec']

#### traitement des generateurs d'ecs par défaut

In [1028]:
gen_ecs_lib_defaut = gen_ecs_lib.query('priorite=="defaut"')

gen_ecs_lib_defaut = gen_ecs_lib_defaut.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ecs_lib_defaut = gen_ecs_lib_defaut.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ecs_lib_defaut=gen_ecs_lib_defaut.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ecs_lib_defaut = gen_ecs_lib_defaut.to_frame('gen_ecs_lib_final_defaut')

td001_sys = td001_sys.merge(gen_ecs_lib_defaut.reset_index(),on='td001_dpe_id',how='left')

null=td001_sys.gen_ecs_lib_final.isnull()

td001_sys.loc[null,'gen_ecs_lib_final']=td001_sys.loc[null,'gen_ecs_lib_final_defaut']

####  ecs solaire

In [1029]:
gen_ecs_lib_solaire = gen_ecs_lib.query('priorite=="solaire"').copy()

gen_ecs_lib_solaire.label = pd.Categorical(gen_ecs_lib_solaire.label,['abscence ecs solaire','ecs solaire'],ordered=True)

gen_ecs_lib_solaire =gen_ecs_lib_solaire.sort_values('label',ascending=True).drop_duplicates(subset='td001_dpe_id')

gen_ecs_lib_solaire=gen_ecs_lib_solaire.loc[gen_ecs_lib_solaire.label=='ecs solaire']

is_ecs_solaire_txt=(gen_ecs_lib_solaire.groupby('td001_dpe_id').label.count().astype(float)>0)

is_ecs_solaire_data=td014_p.groupby('td001_dpe_id').is_ecs_solaire.max().astype(float)>0

td001_sys = td001_sys.merge(is_ecs_solaire_txt.to_frame('is_ecs_solaire_txt'),how='left',on='td001_dpe_id')
td001_sys = td001_sys.merge(is_ecs_solaire_data.to_frame('is_ecs_solaire_data'),how='left',on='td001_dpe_id')


td001_sys['is_ecs_solaire'] = td001_sys.is_ecs_solaire_txt.fillna(0).astype(float)+td001_sys.is_ecs_solaire_data.fillna(0).astype(float)
td001_sys['is_ecs_solaire'] = td001_sys['is_ecs_solaire'] > 0




is_ecs_solaire = td001_sys.is_ecs_solaire==True

td001_sys.loc[is_ecs_solaire,'gen_ecs_lib_final']='ecs solaire + '+td001_sys.loc[is_ecs_solaire,'gen_ecs_lib_final']

#### redressement combinaison mixte indetermine et energie de combustion

In [1030]:
is_production_mixte = td001_sys.gen_ecs_lib_final=="production mixte indetermine"
for ener in energie_chaudiere_mods:
    is_ener = td001_sys.gen_ecs_lib_final_defaut.str.contains(f'ecs {ener} indetermine')
    td001_sys.loc[is_production_mixte&is_ener,'gen_ecs_lib_final']=f'chaudiere {ener} indetermine'

In [1031]:
is_production_mixte = td001_sys.gen_ecs_lib_final=="production mixte indetermine"

is_ener = td001_sys.gen_ecs_lib_final_defaut.str.contains('ecs bois indetermine')
td001_sys.loc[is_production_mixte&is_ener,'gen_ecs_lib_final']=f'chaudiere bois'

####  installation ecs

In [1032]:
type_installation_ecs = type_installation_ecs.sort_values(by=['td001_dpe_id','is_indetermine',
                                     'source','label'])
type_installation_ecs = type_installation_ecs.drop_duplicates(subset=['td001_dpe_id'])

type_installation_ecs = type_installation_ecs.set_index('td001_dpe_id').label.to_frame('type_installation_ecs')

td001_sys = td001_sys.merge(type_installation_ecs.reset_index(),on='td001_dpe_id',how='left')


####  energie ecs

In [1033]:

energie_ecs = energie_ecs.sort_values(by=['td001_dpe_id','label','source'])

energie_ecs = energie_ecs.drop_duplicates(subset=['td001_dpe_id','label'])

energie_ecs=energie_ecs.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x))).to_frame('type_energie_ecs')




td001_sys = td001_sys.merge(energie_ecs.reset_index(),on='td001_dpe_id',how='left')


# Redressement des données 

In [1034]:
td001_sys['gen_ch_lib_final']=td001_sys.gen_ch_lib_final.str.replace(' \+ indetermine','')
td001_sys['gen_ecs_lib_final']=td001_sys.gen_ecs_lib_final.str.replace(' \+ indetermine','')

In [1035]:
td001_sys['gen_ch_lib_non_retraite']=td001_sys.gen_ch_lib_final.copy()
td001_sys['gen_ecs_lib_non_retraite']=td001_sys.gen_ecs_lib_final.copy()

In [1036]:
#td001_sys['gen_ch_lib_final']=td001_sys.gen_ch_lib_non_retraite
#td001_sys['gen_ecs_lib_final']=td001_sys.gen_ecs_lib_non_retraite

## retraitement des energies non affectées pour les chaudières avec les energies factures

In [1037]:
td001_sys =td001_sys.merge(td001_ener_from_facture,on='td001_dpe_id',how='left')


### chauffage

In [1209]:


ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')

for energie in energie_chaudiere_mods:
    
    # ON AFFECTE LES ENERGIES MANQUANTES AVEC LES ENERGIES DE CHAUFFAGE ISSUES DIFFERENTES SOURCES
    
    ener = td001_sys.td002_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.td016_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    # SINON ON PREND LES ENERGIES D'ECS ISSUES DES DIFFERENTES SOURCES

    ener = td001_sys.type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.td002_type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.td016_type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')

    
# Gestion des poeles indetermine
poele_ind = td001_sys.gen_ch_lib_final.str.contains('poele ou insert indetermine')

bois = td001_sys.td002_type_energie_ch.str.contains('bois')
td001_sys.loc[bois&poele_ind,'gen_ch_lib_final']=td001_sys.loc[bois&poele_ind,'gen_ch_lib_final'].str.replace('poele ou insert indetermine',
                                                                                                        'poele ou insert bois')
bois = td001_sys.td016_type_energie_ch.str.contains('bois')
td001_sys.loc[bois&poele_ind,'gen_ch_lib_final']=td001_sys.loc[bois&poele_ind,'gen_ch_lib_final'].str.replace('poele ou insert indetermine',
                                                                                                                          'poele ou insert bois')

fioul = td001_sys.td002_type_energie_ch.str.contains('fioul')
td001_sys.loc[fioul&poele_ind,'gen_ch_lib_final']=td001_sys.loc[fioul&poele_ind,'gen_ch_lib_final'].str.replace('poele ou insert indetermine',
                                                                                                                         'poele ou insert fioul/gpl/charbon')
fioul = td001_sys.td016_type_energie_ch.str.contains('fioul')
td001_sys.loc[fioul&poele_ind,'gen_ch_lib_final']=td001_sys.loc[fioul&poele_ind,'gen_ch_lib_final'].str.replace('poele ou insert indetermine',
                                                                                                                'poele ou insert fioul/gpl/charbon')



### ECS

In [1039]:


ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')

for energie in energie_chaudiere_mods:
    
    # ON AFFECTE LES ENERGIES MANQUANTES AVEC LES ENERGIES D'ECS ISSUES DIFFERENTES SOURCES

    ener = td001_sys.td002_type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    ener = td001_sys.td016_type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    # SINON ON PREND LES ENERGIES DE CHAUFFAGE ISSUES DES DIFFERENTES SOURCES

    ener = td001_sys.type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.td002_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.td016_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')


## cohérence sys ecs chauffage

In [1040]:
td001_sys[['gen_ch_lib_final','gen_ecs_lib_final']]=td001_sys[['gen_ch_lib_final','gen_ecs_lib_final']].fillna('indetermine')

In [1041]:
is_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine')
is_ind_ecs = is_ind_ecs | (td001_sys.gen_ecs_lib_final=="indetermine")
is_ind_ch = td001_sys.gen_ch_lib_final=="indetermine"

### chaudieres

In [1042]:
is_chaudiere_ener_ind_ch = td001_sys.gen_ch_lib_final.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('chaudiere energie indetermine')
is_chaudiere_ch = td001_sys.gen_ch_lib_final.str.contains('chaudiere')
is_chaudiere_ecs = td001_sys.gen_ecs_lib_final.str.contains('chaudiere')



#### affectation du système mixte à l'ECS lorsque celle ci est indetermine

In [1043]:
sub_by_ch=is_ind_ecs&is_chaudiere_ch

In [1044]:
td001_sys.loc[sub_by_ch,'gen_ecs_lib_final']=td001_sys.loc[sub_by_ch,'gen_ch_lib_final']

#### correction et convergence des energies de chaudieres entre ECS et chauffage

In [1045]:
for energie in list(reversed(energie_chaudiere_mods)):
    is_ener_ch = (td001_sys.gen_ch_lib_final.str.contains(energie))
    is_ener_ecs = (td001_sys.gen_ecs_lib_final.str.contains(energie))
    sub_ch_ecs = is_ener_ch&(~is_ener_ecs)&is_chaudiere_ch&is_chaudiere_ecs
    td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final']=td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    sub_ecs_ch = is_ener_ecs&(~is_ener_ch)&is_chaudiere_ch&is_chaudiere_ecs
    td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final']=td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final'].str.replace('energie indetermine',energie)


#### correction et convergence des types de chaudieres entre ECS et chauffage

In [1046]:
for energie in reversed(energie_chaudiere_mods):
    is_ener_ch = (td001_sys.gen_ch_lib_final.str.contains(energie))
    is_ener_ecs = (td001_sys.gen_ecs_lib_final.str.contains(energie))
    both_ener = is_ener_ch&is_ener_ecs&is_chaudiere_ch&is_chaudiere_ecs
    for type_chaudiere in reversed(type_chaudiere_mods):
        is_type_ch = (td001_sys.gen_ch_lib_final.str.contains(type_chaudiere))
        is_type_ecs = (td001_sys.gen_ecs_lib_final.str.contains(type_chaudiere))
        chaudiere_ind = f'chaudiere {energie} indetermine'
        chaudiere_type=f'chaudiere {energie} {type_chaudiere}'
        is_ind_ch = (td001_sys.gen_ch_lib_final.str.contains(chaudiere_ind))
        is_ind_ecs = (td001_sys.gen_ecs_lib_final.str.contains(chaudiere_ind))
        sub_ch_ecs = is_type_ch&both_ener&is_ind_ecs
        td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final']=td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final'].str.replace(chaudiere_ind,chaudiere_type)
        sub_ecs_ch = is_type_ecs&both_ener&is_ind_ch
        td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final']=td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final'].str.replace(chaudiere_ind,chaudiere_type)

        

#### correction cas ECS affecté à un chauffe eau indépendant lorsque chaudiere chauffage

In [1047]:
# refresh
is_chaudiere_ch = td001_sys.gen_ch_lib_final.str.contains('chaudiere')
is_ind = td001_sys.gen_ecs_lib_final.str.contains('chauffe-eau independant indetermine')
is_ind = is_ind |(td001_sys.gen_ecs_lib_final=='indetermine')

In [1048]:
td001_sys.loc[is_chaudiere_ch&is_ind,'gen_ecs_lib_final']=td001_sys.loc[is_chaudiere_ch&is_ind,'gen_ch_lib_final']

#### correction ecs partie chaudiere -> affecté au chauffage par défaut

In [1161]:
is_chaudiere_ecs_and_ch = is_chaudiere_ecs& is_chaudiere_ch

In [1162]:
td001_sys.loc[is_chaudiere_ecs_and_ch,'chaudiere_ecs']=td001_sys.loc[is_chaudiere_ecs_and_ch,'gen_ecs_lib_final'].apply(lambda x:' + '.join([el for el in x.split(' + ') if 'chaudiere' in el]))
td001_sys.loc[is_chaudiere_ecs_and_ch,'chaudiere_ch']=td001_sys.loc[is_chaudiere_ecs_and_ch,'gen_ch_lib_final'].apply(lambda x:' + '.join([el for el in x.split(' + ') if 'chaudiere' in el]))

In [1163]:
td001_sys.loc[is_chaudiere_ecs_and_ch,'not_chaudiere_ecs']=td001_sys.loc[is_chaudiere_ecs_and_ch,'gen_ecs_lib_final'].apply(lambda x:' + '.join([el for el in x.split(' + ') if 'chaudiere' not in el]))
td001_sys.loc[is_chaudiere_ecs_and_ch,'not_chaudiere_ch']=td001_sys.loc[is_chaudiere_ecs_and_ch,'gen_ch_lib_final'].apply(lambda x:' + '.join([el for el in x.split(' + ') if 'chaudiere' not in el]))

In [1166]:
sel=(td001_sys['chaudiere_ch']!=td001_sys.chaudiere_ecs)&is_chaudiere_ecs_and_ch

In [1178]:
td001_sys.loc[sel,"chaudiere_ecs"]=td001_sys.loc[sel,'chaudiere_ch']

In [1180]:
td001_sys.loc[sel,'gen_ecs_lib_final']=td001_sys.loc[sel,'chaudiere_ecs']+td001_sys.loc[sel,'not_chaudiere_ecs']

#### correction cas ecs si gen_ch_lib elec ou poele

In [1049]:
# si le generateur de chauffage est décentralisé et avec une energie non combustible. 
is_ind = td001_sys.gen_ecs_lib_final.str.contains('chauffe-eau independant indetermine')
is_ind = is_ind |(td001_sys.gen_ecs_lib_final=='indetermine')
is_ind = is_ind |(td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine'))

In [1050]:
is_ch_decentr = ~td001_sys.gen_ch_lib_final.str.contains('|'.join(energie_chaudiere_mods))
is_ch_decentr = is_ch_decentr& td001_sys.gen_ch_lib_final.str.contains('elec|pac|joule|jonction|poele')

In [1051]:
td001_sys.loc[is_ind&is_ch_decentr,'gen_ecs_lib_final']='ecs electrique indetermine'

In [1052]:
is_ind =(td001_sys.gen_ecs_lib_final=='indetermine')

### reseaux de chaleur

In [1053]:
# refresh
# si le chauffage ou l'ECS sont indéterminés et que l'autre est un réseau de chaleur alors on substitue par réseau de chaleur
is_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine')
is_ind_ecs = is_ind_ecs | (td001_sys.gen_ecs_lib_final=="indetermine")
is_ind_ch = td001_sys.gen_ch_lib_final=="indetermine"

In [1054]:
is_reseau_ch = td001_sys.gen_ch_lib_final.str.contains('reseau de chaleur')
is_reseau_ecs = td001_sys.gen_ecs_lib_final.str.contains('reseau de chaleur')

td001_sys.loc[is_ind_ch&is_reseau_ecs,'gen_ch_lib_final']='reseau de chaleur'
td001_sys.loc[is_ind_ecs&is_reseau_ch,'gen_ecs_lib_final']='reseau de chaleur'

### thermodynamique

In [1055]:
# refresh

# si l'un est thermodynamique et l'autre indeterminé on considère l'autre comme thermodynamique. 
is_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine')
is_ind_ecs = is_ind_ecs | (td001_sys.gen_ecs_lib_final=="indetermine")
is_ind_ch = td001_sys.gen_ch_lib_final=="indetermine"

In [1056]:
is_pac_ch = td001_sys.gen_ch_lib_final.str.contains('pac')&(~td001_sys.gen_ch_lib_final.str.contains('pac air/air'))

td001_sys.loc[is_ind_ecs&is_pac_ch,'gen_ecs_lib_final']='ecs thermodynamique electrique(pac ou ballon)'

## pac en releve de chaudiere

In [1057]:
# si on a une chaudiere et une pac alors on considère pac en relève de chaudière
is_pac_ch = td001_sys.gen_ch_lib_final.str.contains('pac')&(~td001_sys.gen_ch_lib_final.str.contains('pac air/air'))
is_pac_ch = is_pac_ch& (td001_sys.gen_ch_lib_final !='pac indetermine')
is_chaudiere_ch = td001_sys.gen_ch_lib_final.str.contains('chaudiere|chauffage gaz indetermine')

for pac_type in ['pac air/eau','pac eau/eau','pac geothermique']:
    
    is_pac_ch = td001_sys.gen_ch_lib_final.str.contains(pac_type)
    
    td001_sys.loc[is_pac_ch&is_chaudiere_ch,'gen_ch_lib_final']=f'{pac_type} en releve de chaudiere'
    

## redressement chaudieres elec et bois

In [1058]:
is_chaudiere_elec=td001_sys.gen_ch_lib_final.str.contains('chaudiere energie indetermine indetermine + chauffage electrique indetermine')

td001_sys.loc[is_chaudiere_elec,'gen_ch_lib_final']=td001_sys.loc[is_chaudiere_elec,'gen_ch_lib_final'].str.replace('chaudiere energie indetermine indetermine + chauffage electrique indetermine','chaudiere electrique')

is_chaudiere_bois=td001_sys.gen_ch_lib_final.str.contains('chaudiere bois')
is_chaudiere_ind=td001_sys.gen_ch_lib_final.str.contains('chaudiere energie indetermine indetermine')

td001_sys.loc[is_chaudiere_bois&is_chaudiere_ind,'gen_ch_lib_final']=f'chaudiere bois'

for type_chaudiere in type_chaudiere_mods:
    
    is_chaudiere_ind=td001_sys.gen_ch_lib_final.str.contains(f'chaudiere energie indetermine {type_chaudiere}')

    td001_sys.loc[is_chaudiere_bois&is_chaudiere_ind,'gen_ch_lib_final']=f'chaudiere bois'

In [1059]:
is_chaudiere_elec=td001_sys.gen_ecs_lib_final.str.contains('chaudiere energie indetermine indetermine + chauffage electrique indetermine')

td001_sys.loc[is_chaudiere_elec,'gen_ecs_lib_final']=td001_sys.loc[is_chaudiere_elec,'gen_ecs_lib_final'].str.replace('chaudiere energie indetermine indetermine + chauffage electrique indetermine','chaudiere electrique')

is_chaudiere_bois=td001_sys.gen_ecs_lib_final.str.contains('chaudiere bois')
is_chaudiere_ind=td001_sys.gen_ecs_lib_final.str.contains('chaudiere energie indetermine indetermine')

td001_sys.loc[is_chaudiere_bois&is_chaudiere_ind,'gen_ecs_lib_final']=f'chaudiere bois'

for type_chaudiere in type_chaudiere_mods:
    
    is_chaudiere_ind=td001_sys.gen_ecs_lib_final.str.contains(f'chaudiere energie indetermine {type_chaudiere}')

    td001_sys.loc[is_chaudiere_bois,'gen_ecs_lib_final']=f'chaudiere bois'

## suppression des libéllés chauffage indetermine avec une chaudière

pour résoudre les problèmes de type chaudiere gaz ou fioul -> attribué chaudiere gaz par défaut

In [1060]:
for ener in energie_chaudiere_mods:
    is_chaudiere_ener = td001_sys.gen_ch_lib_final.str.contains(f'chaudiere {ener}')
    for ener_other in energie_chaudiere_mods:
        lib_ind_other=f"chauffage {ener_other} indetermine"
        is_ind_other = td001_sys.gen_ch_lib_final.str.contains(lib_ind_other)
        
        td001_sys.loc[is_chaudiere_ener&is_ind_other,'gen_ch_lib_final']=td001_sys.loc[is_chaudiere_ener&is_ind_other,'gen_ch_lib_final'].str.replace(' \+ '+lib_ind_other,'')


## correction libellés ecs (quand substitué par chauffage)



In [1061]:
ecs_lib_cat = list(gen_ecs_search_dict_flat.keys())+['indetermine']

td001_sys['gen_ecs_lib_final']=td001_sys.gen_ecs_lib_final.apply(lambda x:' + '.join([el for el in x.split(' + ') if el in ecs_lib_cat]))

## affectation de libéllés par défaut en fonction des energies de chauffage et ecs pour les générateurs indéterminés

In [1062]:
ind = td001_sys.gen_ch_lib_final=="indetermine"

for type_ener in td001_sys.td002_type_energie_ch.dropna().unique():
    
    ener = td001_sys.td002_type_energie_ch==type_ener
    
    if type_ener in energie_chaudiere_mods:
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=f'chaudiere {type_ener} indetermine'
        
    elif 'reseau de chaleur' in type_ener : 
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=f'reseau de chaleur'
        
    elif 'electricite' == type_ener : 
        td001_sys.loc[ind&ener,'gen_ch_lib_final']='chauffage electrique indetermine'

    else:
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=' + '.join([f'chauffage {el} indetermine' for el in type_ener.split(' + ')])

for type_ener in td001_sys.td016_type_energie_ch.dropna().unique():

    ener = td001_sys.td016_type_energie_ch==type_ener
    
    if type_ener in energie_chaudiere_mods:
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=f'chaudiere {type_ener} indetermine'
        
    elif 'reseau de chaleur' in type_ener : 
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=f'reseau de chaleur'
        
    elif 'electricite' == type_ener : 
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=f'chauffage electrique indetermine'   
        
    else:
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=' + '.join([f'chauffage {el} indetermine' for el in type_ener.split(' + ')])

td001_sys.gen_ch_lib_final = td001_sys.gen_ch_lib_final.str.replace('chauffage electricite','chauffage electrique')


In [1063]:
ind = td001_sys.gen_ecs_lib_final=="indetermine"

for type_ener in td001_sys.td002_type_energie_ecs.dropna().unique():
    
    ener = td001_sys.td002_type_energie_ecs==type_ener
    
    if type_ener in energie_chaudiere_mods:
        
        td001_sys.loc[ind&ener,'gen_ecs_lib_final']=f'chaudiere {type_ener} indetermine'
        
    elif 'reseau de chaleur' in type_ener : 
        
        td001_sys.loc[ind&ener,'gen_ecs_lib_final']=f'reseau de chaleur'
        
    elif 'electricite' == type_ener : 
        td001_sys.loc[ind&ener,'gen_ecs_lib_final']='ecs electrique indetermine'

    else:
        td001_sys.loc[ind&ener,'gen_ecs_lib_final']=' + '.join([f'ecs {el} indetermine' for el in type_ener.split(' + ')])

for type_ener in td001_sys.td016_type_energie_ecs.dropna().unique():

    ener = td001_sys.td016_type_energie_ecs==type_ener
    
    if type_ener in energie_chaudiere_mods:
        
        td001_sys.loc[ind&ener,'gen_ecs_lib_final']=f'chaudiere {type_ener} indetermine'
        
    elif 'reseau de chaleur' in type_ener : 
        
        td001_sys.loc[ind&ener,'gen_ecs_lib_final']=f'reseau de chaleur'
        
    elif 'electricite' == type_ener : 
        
        td001_sys.loc[ind&ener,'gen_ecs_lib_final']=f'ecs electrique indetermine'   
        
    else:
        
        td001_sys.loc[ind&ener,'gen_ecs_lib_final']=' + '.join([f'ecs {el} indetermine' for el in type_ener.split(' + ')])

td001_sys.gen_ecs_lib_final = td001_sys.gen_ecs_lib_final.str.replace('ecs electricite','ecs electrique')

In [1064]:
td001_sys['concat']=td001_sys.gen_ch_lib_final+'////'+td001_sys.gen_ecs_lib_final

## reorder labels

In [1065]:
ordered_ch_labels = ['chauffage solaire']+list(gen_ch_search_dict_flat.keys())+['chauffage autre indetermine','indetermine']


In [1066]:
ordered_ecs_labels =list(gen_ecs_search_dict_flat.keys())+['ecs autre indetermine','indetermine']


In [1067]:
def resort_gen_ch_labels(x):
    try:
        c=pd.Categorical(x.split(' + '),ordered_ch_labels,ordered=True)
        c = ' + '.join(c.sort_values().tolist())
        return c
    except Exception as e:
        print(x)
        print(c)
        raise e
def resort_gen_ecs_labels(x):
    try:
        c=pd.Categorical(x.split(' + '),ordered_ecs_labels,ordered=True)
        c = ' + '.join(c.sort_values().tolist())
    except Exception as e:
        print(x)
        print(c)
        raise e
    return x

In [1068]:
plus = td001_sys.gen_ch_lib_final.str.contains(' \+ ')

td001_sys.loc[plus,'gen_ch_lib_final'] = td001_sys.loc[plus,'gen_ch_lib_final'].apply(lambda x:resort_gen_ch_labels(x))

plus = td001_sys.gen_ecs_lib_final.str.contains(' \+ ')

td001_sys.loc[plus,'gen_ecs_lib_final'] = td001_sys.loc[plus,'gen_ecs_lib_final'].apply(lambda x:resort_gen_ecs_labels(x))

## redressement energie 

### chauffage

In [1118]:
gen_to_energy = dict()

for k in ordered_ch_labels:
    ener = energie_normalise_ordered
    for ener in reversed(energie_normalise_ordered):
        if ener in k:
            gen_to_energy[k]=ener
    if 'pac' in k or 'elec' in k :
        gen_to_energy[k]='electricite'
    if k not in gen_to_energy:
        gen_to_energy[k]=''
gen_to_energy['poele ou insert fioul/gpl/charbon']=''
gen_to_energy['poele ou insert indetermine']=''

td001_sys['type_energie_ch_from_gen']=td001_sys.gen_ch_lib_final.copy()
for k,v in gen_to_energy.items():
    td001_sys['type_energie_ch_from_gen']=td001_sys['type_energie_ch_from_gen'].str.replace(k,v)

td001_sys['type_energie_ch_from_gen']=td001_sys['type_energie_ch_from_gen'].apply(lambda x:' + '.join(np.unique([el for el in x.split(' + ') if el!=''])))

null=td001_sys.type_energie_ch.isnull()

td001_sys.loc[null,'type_energie_ch']=td001_sys.loc[null,'type_energie_ch_from_gen']

is_releve = td001_sys.gen_ch_lib_final.str.contains('releve')
td001_sys.loc[is_releve,'type_energie_ch']=td001_sys.loc[is_releve,'type_energie_ch']+' + '+td001_sys.loc[is_releve,'type_energie_ch_from_gen']

td001_sys.loc[is_releve,'type_energie_ch'] = td001_sys.loc[is_releve,'type_energie_ch'].apply(lambda x:' + '.join(np.unique([el for el in x.split(' + ')]))) 

bad_corr=td001_sys.type_energie_ch!=td001_sys.type_energie_ch_from_gen
gpl_gaz_mixup=td001_sys.type_energie_ch.str.contains('gpl/butane/propane')&bad
gpl_gaz_mixup=gpl_gaz_mixup&td001_sys.type_energie_ch_from_gen.str.contains('gaz')
td001_sys.loc[gpl_gaz_mixup,'type_energie_ch']=td001_sys.loc[gpl_gaz_mixup,'type_energie_ch'].str.replace('gpl/butane/propane','gaz')

td001_sys.loc[gpl_gaz_mixup,'type_energie_ch']=td001_sys.loc[gpl_gaz_mixup,'type_energie_ch'].apply(lambda x:' + '.join(np.unique([el for el in x.split(' + ')]))) 

bad_corr=td001_sys.type_energie_ch!=td001_sys.type_energie_ch_from_gen


inf=td001_sys[bad].type_energie_ch.str.count('\+')<td001_sys[bad].type_energie_ch_from_gen.str.count('\+')
inf=inf[inf].index
td001_sys.loc[inf,'type_energie_ch']=td001_sys.loc[inf,'type_energie_ch_from_gen']

### ecs

In [1119]:
gen_to_energy = dict()

for k in ordered_ecs_labels:
    ener = energie_normalise_ordered
    for ener in reversed(energie_normalise_ordered):
        if ener in k:
            gen_to_energy[k]=ener
    if 'pac' in k or 'elec' in k :
        gen_to_energy[k]='electricite'
    if k not in gen_to_energy:
        gen_to_energy[k]=''

In [1122]:


td001_sys['type_energie_ecs_from_gen']=td001_sys.gen_ecs_lib_final.copy()
for k,v in gen_to_energy.items():
    td001_sys['type_energie_ecs_from_gen']=td001_sys['type_energie_ecs_from_gen'].str.replace(k,v)

td001_sys['type_energie_ecs_from_gen']=td001_sys['type_energie_ecs_from_gen'].apply(lambda x:' + '.join(np.unique([el for el in x.split(' + ') if el!=''])))


In [1123]:

null=td001_sys.type_energie_ecs.isnull()

td001_sys.loc[null,'type_energie_ecs']=td001_sys.loc[null,'type_energie_ecs_from_gen']

In [1124]:
bad_corr=td001_sys.type_energie_ecs!=td001_sys.type_energie_ecs_from_gen


In [1129]:
td001_sys.loc[bad_corr,['td001_dpe_id','gen_ecs_lib_final','gen_ecs_lib_infer_concat']]

,td001_dpe_id,gen_ecs_lib_final,gen_ecs_lib_infer_concat
32,4030,chaudiere gaz indetermine,chaudiere gaz indetermine
42,4728,chaudiere bois,ecs gaz indetermine
44,4945,ecs solaire + ecs thermodynamique electrique(p...,ecs thermodynamique electrique(pac ou ballon)
50,5460,chaudiere bois,ecs gaz indetermine
57,6035,chaudiere bois,ecs fioul indetermine
...,...,...,...
170199,9428755,chaudiere gaz indetermine,chaudiere gaz indetermine
170205,9429469,ecs thermodynamique electrique(pac ou ballon),NaN
170210,9429816,ecs thermodynamique electrique(pac ou ballon),ballon a accumulation electrique + ecs thermod...
170220,9430624,chaudiere bois,ecs fioul indetermine


In [ ]:


is_releve = td001_sys.gen_ecs_lib_final.str.contains('releve')
td001_sys.loc[is_releve,'type_energie_ecs']=td001_sys.loc[is_releve,'type_energie_ecs']+' + '+td001_sys.loc[is_releve,'type_energie_ecs_from_gen']

td001_sys.loc[is_releve,'type_energie_ecs'] = td001_sys.loc[is_releve,'type_energie_ecs'].apply(lambda x:' + '.join(np.unique([el for el in x.split(' + ')]))) 

bad_corr=td001_sys.type_energie_ecs!=td001_sys.type_energie_ecs_from_gen
gpl_gaz_mixup=td001_sys.type_energie_ecs.str.contains('gpl/butane/propane')&bad
gpl_gaz_mixup=gpl_gaz_mixup&td001_sys.type_energie_ecs_from_gen.str.contains('gaz')
td001_sys.loc[gpl_gaz_mixup,'type_energie_ecs']=td001_sys.loc[gpl_gaz_mixup,'type_energie_ecs'].str.replace('gpl/butane/propane','gaz')

td001_sys.loc[gpl_gaz_mixup,'type_energie_ecs']=td001_sys.loc[gpl_gaz_mixup,'type_energie_ecs'].apply(lambda x:' + '.join(np.unique([el for el in x.split(' + ')]))) 

bad_corr=td001_sys.type_energie_ecs!=td001_sys.type_energie_ecs_from_gen


inf=td001_sys[bad].type_energie_ecs.str.count('\+')<td001_sys[bad].type_energie_ecs_from_gen.str.count('\+')
inf=inf[inf].index
td001_sys.loc[inf,'type_energie_ecs']=td001_sys.loc[inf,'type_energie_ecs_from_gen']

# indicateurs de cohérences

In [ ]:
lib_final_eq_lib_data = (td001_sys.gen_ch_lib_infer_concat==td001_sys.gen_ch_lib_final).fillna(False)

td001_sys['fiab_gen_ch_lib'] = 'libelle non redresse determine'

ind_data=td001_sys.gen_ch_lib_infer_concat.str.contains('indetermine').fillna(False)
not_data = td001_sys.gen_ch_lib_infer_concat.isnull()
td001_sys.loc[lib_final_eq_lib_data&(~ind_data),'fiab_gen_ch_lib']='libelle fiable et coherent ft desc et td012'

is_redresse = td001_sys.gen_ch_lib_final!=td001_sys.gen_ch_lib_non_retraite
td001_sys.loc[is_redresse,'fiab_gen_ch_lib'] = 'libelle redresse determine'

is_ind = td001_sys.gen_ch_lib_final=='indetermine'

td001_sys.loc[is_ind,'fiab_gen_ch_lib'] = 'libelle indetermine'

is_ind = td001_sys.gen_ch_lib_final.str.contains('indetermine')

td001_sys.loc[is_ind,'fiab_gen_ch_lib'] = td001_sys.loc[is_ind,'fiab_gen_ch_lib'].str.replace(' determine',' partiellement indetermine')



In [ ]:
is_ind = td001_sys.gen_ecs_lib_final.str.contains('indetermine')


In [ ]:
lib_final_eq_lib_data = (td001_sys.gen_ecs_lib_infer_concat==td001_sys.gen_ecs_lib_final).fillna(False)

td001_sys['fiab_gen_ecs_lib'] = 'libelle non redresse determine'

ind_data=td001_sys.gen_ecs_lib_infer_concat.str.contains('indetermine').fillna(False)
not_data = td001_sys.gen_ecs_lib_infer_concat.isnull()
td001_sys.loc[lib_final_eq_lib_data&(~ind_data),'fiab_gen_ecs_lib']='libelle fiable et coherent ft desc et td012'

is_redresse = td001_sys.gen_ecs_lib_final!=td001_sys.gen_ecs_lib_non_retraite
td001_sys.loc[is_redresse,'fiab_gen_ecs_lib'] = 'libelle redresse determine'

is_ind = td001_sys.gen_ecs_lib_final=='indetermine'

td001_sys.loc[is_ind,'fiab_gen_ecs_lib'] = 'libelle indetermine'

is_ind = td001_sys.gen_ecs_lib_final.str.contains('indetermine')

td001_sys.loc[is_ind,'fiab_gen_ecs_lib'] = td001_sys.loc[is_ind,'fiab_gen_ecs_lib'].str.replace(' determine',' partiellement indetermine')



# nettoyage final

## suppression des additions d'indeterminé

# simplification 

## sys ch

In [ ]:
td001_sys['gen_ch_lib_final_simp']=td001_sys['gen_ch_lib_final']

### simplification des dénominations generateurs

In [ ]:
gen_ch_simp_dict = {
                        'convecteurs electriques nfc': 'generateurs a effet joule',
                        'panneaux rayonnants electriques nfc': 'generateurs a effet joule',
                        'radiateurs electriques': 'generateurs a effet joule',
                        'autres emetteurs a effet joule': 'generateurs a effet joule',
                        'plafonds/planchers rayonnants electriques nfc': 'generateurs a effet joule',
                        'convecteurs bi-jonction': 'generateurs a effet joule',
    "convecteurs electriques":'generateurs a effet joule',
'chauffage electrique indetermine': 'generateurs a effet joule',
}

for ener in energie_chaudiere_mods:
    gen_ch_simp_dict[f'chauffage {ener} indetermine']=f'chaudiere {ener} standard'
    gen_ch_simp_dict[f'chauffage electricite + {ener} indetermine']=f'chaudiere {ener} standard'

    gen_ch_simp_dict[f'chaudiere {ener} indetermine']=f'chaudiere {ener} standard'


In [ ]:
for k,v in gen_ch_simp_dict.items():
    td001_sys.gen_ch_lib_final_simp=td001_sys.gen_ch_lib_final_simp.str.replace(k,v)
    

### simplification réseau de chaleur
si réseau de chaleur alors on considère que c'est le système de chauffage du bâtiment

In [ ]:
is_reseau_ch = td001_sys.gen_ch_lib_final.str.contains('reseau de chaleur')

td001_sys.loc[is_reseau_ch,'gen_ch_lib_final_simp']='reseau de chaleur'

### limiter les combinaisons de systemes à 2

In [ ]:
is_sup_2_sys=td001_sys.gen_ch_lib_final_simp.str.count(' \+ ')>=2

In [ ]:
td001_sys.loc[is_sup_2_sys,'gen_ch_lib_final_simp']=td001_sys.loc[is_sup_2_sys,'gen_ch_lib_final_simp'].apply(lambda x:' + '.join(x.split(' + ')[0:2]))

### simplification de la mention en releve de chaudiere pour les systèmes thermo

on considère les systèmes thermo comme le principal usage.

In [ ]:
td001_sys.gen_ch_lib_final_simp=td001_sys.gen_ch_lib_final_simp.str.replace(' en releve de chaudiere','')


## sys ecs

In [ ]:
td001_sys['gen_ecs_lib_final_simp']=td001_sys['gen_ecs_lib_final']

In [ ]:
gen_ecs_simp_dict = {
                        'convecteurs electriques nfc': 'generateurs a effet joule',
                        'panneaux rayonnants electriques nfc': 'generateurs a effet joule',
                        'radiateurs electriques': 'generateurs a effet joule',
                        'autres emetteurs a effet joule': 'generateurs a effet joule',
                        'plafonds/planchers rayonnants electriques nfc': 'generateurs a effet joule',
                        'convecteurs bi-jonction': 'generateurs a effet joule',
    "convecteurs electriques":'generateurs a effet joule',
'chauffage electrique indetermine': 'generateurs a effet joule',
}



for ener in energie_chaudiere_mods:
    gen_ecs_simp_dict[f'ecs {ener} indetermine']=f'chaudiere {ener} standard'
    gen_ecs_simp_dict[f'ecs electricite + {ener} indetermine']=f'chaudiere {ener} standard'
    gen_ecs_simp_dict[f'production mixte {ener}']=f'chaudiere {ener} standard'


In [ ]:
td001_sys.gen_ecs_lib_final.value_counts().to_dict()

# debug

In [ ]:
ids=td001_sys.query('gen_ecs_lib_final=="chaudiere energie indetermine standard + reseau de chaleur"').td001_dpe_id.unique()

gen_ecs_lib_from_txt=pd.concat([gen_ecs_lib_desc[['label','category','td001_dpe_id']],
                               gen_ecs_lib_ft[['label','category','td001_dpe_id']]],axis=0)




m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ecs_search_dict_flat)
m = categorize_search_res(m,label_cat = list(gen_ecs_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ecs)
gen_ecs_lib_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')

gen_ecs_lib_ft

td001_sys.loc[td001_sys.td001_dpe_id=="8085231"]

gen_ecs_lib_ft.loc[gen_ecs_lib_ft.td001_dpe_id=='8085231']

gen_ecs_lib_from_txt.loc[gen_ecs_lib_from_txt.td001_dpe_id=='8085231']

gen_ecs_lib.loc[gen_ecs_lib.td001_dpe_id=='8085231']

td005_ch.loc[td005_ch.td001_dpe_id.isin(ids)].to_dict(orient='records')

td003_ch.loc[td003_ch.td001_dpe_id.isin(ids)].to_dict(orient='records')

td003_ecs.loc[td003_ecs.td001_dpe_id.isin(ids)].to_dict(orient='records')

td005_ecs.loc[td005_ecs.td001_dpe_id.isin(ids)].to_dict(orient='records')

td001_sys.gen_ecs_lib_final.fillna('NA').value_counts()

In [ ]:
ind = td001_sys.gen_ch_lib_final.str.contains('indetermine')

In [ ]:
ind_dict=td001_sys.loc[ind,'gen_ch_lib_final'].value_counts().to_dict()

In [1104]:
dpe_list=td001_sys.loc[td001_sys.gen_ch_lib_final=='pac eau/eau'].td001_dpe_id.tolist()

In [ ]:
dpe_list = gen_ecs_lib.loc[gen_ecs_lib.label=="ecs bois indetermine"].td001_dpe_id.tolist()

In [ ]:
dpe_list=td001_sys.loc[td001_sys.gen_ch_lib_final_simp== 'chaudiere energie indetermine standard'].td001_dpe_id.tolist()

In [ ]:
dpe_list=gen_ecs_lib.loc[gen_ecs_lib.label== 'ecs gaz indetermine'].td001_dpe_id.tolist()

In [ ]:
ind_keys = list(ind_dict.keys())

In [ ]:
ind_key=ind_keys.pop(0)
dpe_list = td001_sys.loc[td001_sys.gen_ch_lib_final==  ind_key].td001_dpe_id.tolist()

In [ ]:
ind_key

In [ ]:
gen_ecs_lib.loc[gen_ecs_lib.td001_dpe_id=="137882"]

In [1106]:
"pac eau  eau sur plancher et radiateurs.".strip()

'pac eau  eau sur plancher et radiateurs.'

In [1105]:
a_list =td003_ch.loc[td003_ch.td001_dpe_id.isin(dpe_list)].descriptif.tolist()

words = list()
for el in a_list:
    words.extend(el.split(' '))

pd.Series(words).value_counts().to_dict()

sel=td003_ch.loc[td003_ch.td001_dpe_id.isin(dpe_list)].set_index('td001_dpe_id')
sel.descriptif.to_dict()

{'7243': 'pompe a chaleur eau eau reseau d eau isole \r  emetteurs: planchers chauffants   radiateurs aluminium',
 '360928': 'pompe a chaleur eau eau systeme basse temperature \r emetteurs: planchers chauffants',
 '517207': 'pompe a chaleur eau eau\r ',
 '542426': 'installation de chauffage \r - chauffage 1  pac eau eau  energie electrique  reseau individuel',
 '596369': 'pompe a chaleur eau eau \r emetteurs: radiateurs aluminium munis de robinets thermostatiques',
 '610661': 'pompe a chaleur eau eau\r ',
 '622002': 'pompe a chaleur eau eau reseau d eau isole \r emetteurs: radiateurs munis de robinets thermostatiques',
 '694490': 'installation de chauffage \r - chauffage 1  pac eau eau  energie electrique  reseau individuel',
 '754766': 'pompe a chaleur eau eau \remetteurs: radiateurs',
 '821602': ' - effet joule  pac duo',
 '882965': 'pompe a chaleur eau eau electrique aucun equipement individuel',
 '887382': 'pompe a chaleur eau eau systeme basse temperature  systeme individuel) \r e

In [1107]:
sel=td005_ch.loc[td005_ch.td001_dpe_id.isin(dpe_list)]



In [1108]:
sel.valeur_renseignee.to_dict()

{1521: 'pompe a chaleur eau eau reseau d eau isole \r  emetteurs: planchers chauffants   radiateurs aluminium<br  re : 1  rr : 1  rd : 1  rg : 3.2  pn : 0  fch : 0',
 120697: 'pompe a chaleur eau eau systeme basse temperature \r emetteurs: radiateurs fonte munis de robinets thermostatiques<br  ',
 120698: 'pompe a chaleur eau eau systeme basse temperature \r emetteurs: planchers chauffants<br  ',
 175153: 'installation de chauffage  295\xa0m²)\xa0:\r - chauffage 1 : pac eau eau  energie : electrique)  avec equipement d intermittence central avec minimum de temperature  absence de regulation par piece  reseau de distribution isole ; emetteurs : plancher chauffant avec regulation terminale fabrique s) apres 2000  plancher basse temperature) ',
 190853: 'pompe a chaleur eau eau \r emetteurs: radiateurs aluminium munis de robinets thermostatiques<br  re : 1  rr : 1  rd : 0 9  rg : 3 2  pn : 0  fch : 0',
 199194: 'pompe a chaleur eau eau reseau d eau isole \r emetteurs: radiateurs munis de 

In [563]:
dpe_list=td001_sys.loc[td001_sys.gen_ecs_lib_final==   'ecs thermodynamique electrique(pac ou ballon) + reseau de chaleur'].td001_dpe_id.tolist()

In [ ]:
dpe_list=gen_ecs_lib.loc[gen_ecs_lib.label== 'ecs gaz indetermine'].td001_dpe_id.tolist()

In [564]:
sel=td003_ecs.loc[td003_ecs.td001_dpe_id.isin(dpe_list)].set_index('td001_dpe_id')

sel.descriptif.to_dict()

{'7288192': 'combine au systeme: pompe a chaleur air   air sur reseau de chaleur  systeme collectif)'}

In [567]:
sel=td005_ecs.loc[td005_ecs.td001_dpe_id.isin(dpe_list)].set_index('td001_dpe_id')

sel.valeur_renseignee.to_dict()

{'103575': 'ecs n 1   type d installation : sans ecs solaire   energie : reseau de chaleur urbain   type equipement : reseau de chaleur   contiguite des pieces : pieces contigues   position de la production : en volume non chauffe   isolation du reseau : reseau non isole   volume de stockage : 1000   veilleuse : non   alimentation : heure pleine   ',
 '480207': 'ecs n 1   type d installation : sans ecs solaire   energie : reseau de chaleur urbain   type equipement : reseau de chaleur   contiguite des pieces : pieces non contigues   position de la production : en volume non chauffe   isolation du reseau : reseau isole   volume de stockage : 5000   veilleuse : non   alimentation : heure pleine   ',
 '885202': 'ecs n 1   type d installation : sans ecs solaire   energie : reseau de chaleur urbain   type equipement : reseau de chaleur   contiguite des pieces : pieces contigues   position de la production : en volume non chauffe   isolation du reseau : reseau non isole   volume de stockage :

In [568]:
td001_sys.loc[td001_sys.td001_dpe_id.isin(dpe_list)]

,td001_dpe_id,tr002_type_batiment_id,gen_ch_lib_infer_concat,gen_ecs_lib_infer_concat,gen_ch_lib_final,is_ch_solaire_data,is_ch_solaire_txt,is_ch_solaire,type_installation_ch,type_energie_ch,...,is_ecs_solaire,type_installation_ecs,type_energie_ecs,gen_ch_lib_non_retraite,gen_ecs_lib_non_retraite,td002_type_energie_ch,td002_type_energie_ecs,td016_type_energie_ch,td016_type_energie_ecs,concat
1691,103575,2,reseau de chaleur,ecs thermodynamique electrique(pac ou ballon),reseau de chaleur,False,NaN,False,individuel,reseau de chaleur,...,False,collectif,reseau de chaleur,reseau de chaleur,ecs thermodynamique electrique(pac ou ballon) ...,reseau de chaleur,reseau de chaleur,reseau de chaleur,reseau de chaleur,reseau de chaleur////ecs thermodynamique elect...
8456,480207,2,reseau de chaleur,ecs thermodynamique electrique(pac ou ballon),reseau de chaleur,False,NaN,False,individuel,reseau de chaleur,...,False,collectif,reseau de chaleur,reseau de chaleur,ecs thermodynamique electrique(pac ou ballon) ...,reseau de chaleur,reseau de chaleur,reseau de chaleur,reseau de chaleur,reseau de chaleur////ecs thermodynamique elect...
15714,885202,2,reseau de chaleur,ecs thermodynamique electrique(pac ou ballon),reseau de chaleur + plafonds/planchers rayonna...,False,NaN,False,individuel,reseau de chaleur,...,False,collectif,reseau de chaleur,plafonds/planchers rayonnants electriques nfc ...,ecs thermodynamique electrique(pac ou ballon) ...,reseau de chaleur,reseau de chaleur,reseau de chaleur,reseau de chaleur,plafonds/planchers rayonnants electriques nfc ...
24434,1382671,2,reseau de chaleur,ecs thermodynamique electrique(pac ou ballon),reseau de chaleur,False,NaN,False,individuel,reseau de chaleur,...,False,collectif,reseau de chaleur,reseau de chaleur,ecs thermodynamique electrique(pac ou ballon) ...,reseau de chaleur,NaN,reseau de chaleur,NaN,reseau de chaleur////ecs thermodynamique elect...
42744,2326297,2,reseau de chaleur,ecs thermodynamique electrique(pac ou ballon),reseau de chaleur,False,NaN,False,individuel,reseau de chaleur,...,False,collectif,reseau de chaleur,reseau de chaleur,ecs thermodynamique electrique(pac ou ballon) ...,reseau de chaleur,reseau de chaleur,reseau de chaleur,reseau de chaleur,reseau de chaleur////ecs thermodynamique elect...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129464,7288192,2,NaN,NaN,pac air/air + reseau de chaleur,NaN,NaN,False,collectif,NaN,...,False,collectif,NaN,pac air/air + reseau de chaleur,ecs thermodynamique electrique(pac ou ballon) ...,autre,NaN,NaN,NaN,pac air/air + reseau de chaleur////ecs thermod...
129833,7311563,3,chaudiere bois,ecs thermodynamique electrique(pac ou ballon),chaudiere bois,False,NaN,False,individuel,bois,...,False,individuel,bois,chaudiere bois + chaudiere energie indetermine...,ecs thermodynamique electrique(pac ou ballon) ...,bois,bois,bois,bois,chaudiere bois////ecs thermodynamique electriq...
129886,7314319,2,chaudiere bois,ecs thermodynamique electrique(pac ou ballon),chaudiere bois,False,NaN,False,individuel,bois,...,False,collectif,bois,chaudiere bois + chaudiere energie indetermine...,ecs thermodynamique electrique(pac ou ballon) ...,bois,bois,bois,bois,chaudiere bois////ecs thermodynamique electriq...
148368,8375736,2,reseau de chaleur,ecs thermodynamique electrique(pac ou ballon),reseau de chaleur + plafonds/planchers rayonna...,False,NaN,False,individuel,reseau de chaleur,...,False,collectif,reseau de chaleur,plafonds/planchers rayonnants electriques nfc ...,ecs thermodynamique electrique(pac ou ballon) ...,reseau de chaleur,reseau de chaleur,reseau de chaleur,reseau de chaleur,plafonds/planchers rayonnants electriques nfc ...


# debug ind


In [1146]:
td001_sys.loc[td001_sys.td001_dpe_id=="4728"][['gen_ch_lib_final','gen_ecs_lib_final']]

,gen_ch_lib_final,gen_ecs_lib_final
42,chaudiere gaz standard,chaudiere bois


In [ ]:
gen_ch_lib.loc[gen_ch_lib.td001_dpe_id.isin(dpe_list)]

In [1143]:
gen_ecs_lib_princ

,gen_ecs_lib_final
td001_dpe_id,
1000252,chaudiere gaz standard
1000635,chaudiere fioul indetermine
100076,chaudiere energie indetermine standard
1000917,ecs thermodynamique electrique(pac ou ballon)
100094,chaudiere bois + chaudiere gaz indetermine
...,...
998811,chaudiere gaz condensation
998869,chaudiere energie indetermine indetermine
999090,chaudiere energie indetermine indetermine


In [1169]:
gen_ecs_lib_princ.loc['13978']

gen_ecs_lib_final    chaudiere energie indetermine condensation
Name: 13978, dtype: object

In [1170]:
gen_ecs_lib.loc[gen_ecs_lib.td001_dpe_id=='4728']

,td001_dpe_id,label,category,source,chaudiere_ener_ind,chaudiere_ind,priorite
242174,4728,chaudiere gaz indetermine,chaudiere,txt,False,True,principal
242175,4728,chaudiere bois,chaudiere bois,txt,False,False,principal
242177,4728,ecs fioul indetermine,ecs fioul indetermine,txt,False,False,defaut


In [1171]:
gen_ecs_lib

,td001_dpe_id,label,category,source,chaudiere_ener_ind,chaudiere_ind,priorite
0,291,ballon a accumulation electrique,effet joule,data,False,False,secondaire
1,493,ballon a accumulation electrique,effet joule,data,False,False,secondaire
2,708,ballon a accumulation electrique,effet joule,data,False,False,secondaire
3,1466,chaudiere fioul indetermine,chaudiere,data,False,True,principal
4,1708,ballon a accumulation electrique,effet joule,data,False,False,secondaire
...,...,...,...,...,...,...,...
434915,99972,ballon a accumulation electrique,effet joule,txt,False,False,secondaire
434916,999768,chauffe-eau independant indetermine,chauffe-eau_independant,txt,False,False,defaut
434917,999768,ballon a accumulation electrique,effet joule,txt,False,False,secondaire
434918,999890,chauffe-eau independant indetermine,chauffe-eau_independant,txt,False,False,defaut


In [1172]:
id_="4728"

In [1173]:
td001_sys.query(f'td001_dpe_id=="{id_}"')

,td001_dpe_id,tr002_type_batiment_id,gen_ch_lib_infer_concat,mix_energetique_ch,gen_ecs_lib_infer_concat,gen_ch_lib_final,is_ch_solaire_data,is_ch_solaire_txt,is_ch_solaire,type_installation_ch,...,td002_type_energie_ecs,td016_type_energie_ch,td016_type_energie_ecs,concat,type_energie_ch_from_gen,type_energie_ecs_from_gen,chaudiere_ecs,chaudiere_ch,not_chaudiere_ecs,not_chaudiere_ch
42,4728,1,chaudiere gaz standard,gaz,ecs gaz indetermine,chaudiere gaz standard,False,NaN,False,individuel,...,gaz,NaN,NaN,chaudiere gaz standard////chaudiere bois,gaz,bois,chaudiere bois,chaudiere gaz standard,,


In [1174]:
td005_ecs.loc[td005_ecs.td001_dpe_id==id_].to_dict(orient='records')


[{'id': '91578',
  'td001_dpe_id': '4728',
  'tr011_sous_categorie_fiche_technique_id': '17',
  'valeur_renseignee': 'ecs 1 : production par chaudiere gaz  fioul  bois  0l)',
  'est_efface': '0'}]

In [1175]:
td003_ch.loc[td003_ch.td001_dpe_id==id_].to_dict(orient='records')


[{'id': '55675',
  'td001_dpe_id': '4728',
  'tr007_type_descriptif_id': '11',
  'descriptif': '1 - installation de chauffage sans solaire',
  'est_efface': '0'}]

In [1176]:
td005_ch.loc[td005_ch.td001_dpe_id==id_].to_dict(orient='records')


[{'id': '91577',
  'td001_dpe_id': '4728',
  'tr011_sous_categorie_fiche_technique_id': '16',
  'valeur_renseignee': 'inst 1 : 1 - installation de chauffage sans solaire',
  'est_efface': '0'}]

In [1177]:
td003_ecs.loc[td003_ecs.td001_dpe_id==id_].to_dict(orient='records')


[{'id': '55674',
  'td001_dpe_id': '4728',
  'tr007_type_descriptif_id': '10',
  'descriptif': 'production par chaudiere gaz  fioul  bois',
  'est_efface': '0'}]

In [ ]:
gen_ch_lib_ft.loc[gen_ch_lib_ft.td001_dpe_id==id_]

In [ ]:
m.loc[m.id=='174001369']

In [ ]:
energie_ch_ft.loc[energie_ch_ft.td001_dpe_id==id_]

In [ ]:
m.loc[gen_ch_lib_ft.td001_dpe_id==id_]

In [ ]:
m

In [ ]:
999852

In [ ]:
data_to_search = td005_ch


In [ ]:
id_col='id'
val_col='valeur_renseignee'
search_dict=gen_ch_search_dict_flat

In [ ]:
es_client = setup_es_client()
bulk(es_client, gendata(index_name,data_to_search[id_col],data_to_search[val_col]))
# res_serie = search_from_search_dict(es_client,search_dict,index_name=index_name)

# res_table=res_serie.to_frame('label')

# res_table.index.name = 'id'
# res_table=res_table.reset_index()

In [ ]:
res_table.loc[res_table.id=='174001369']

In [ ]:
s_list = list()
for label,char_list in  search_dict.items():

    search_instruction =generate_instruction_from_list(char_list)
    search_body = {
        "query": {
            "query_string": {
                "query": search_instruction,
                "default_field": "text_to_analyze"
            },

        },

    }
    #es_client.count(index=index_name, body=search_body)

    a_dict=es_client.search(index=index_name, body=search_body,size=500000)

    hits = a_dict['hits']['hits']

    s=pd.Series(index=[el['_source']['id'] for el in hits])
    
    s[:]=label
    s_list.append(s)
s_all=pd.concat(s_list)

In [ ]:
[el['_source'] for el in hits if  el['_source']['id']=='174001369']

In [ ]:
s[:]=label

In [ ]:
s

In [ ]:
[el for el in s_all el =='174001369']

In [ ]:
hits

# debug search

In [ ]:
toto="""
systeme de production d ecs principal surface couverte = 60.0 m2 ancien ballon electrique installe en 2007. les pieces desservies sont contigues. la production est en volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 200 
"""
toto="""
 - collectif : solaire avec appoint chaudiere a condensation gaz naturel"""

toto="""

'type d equipement : chaudiere bois classe 4 energie : bois type d installation : individuelle  sans solaire description : standard  anciennete 0 distribution : production dans le volume chauffe  pieces alimentees contigues  :  '
"""
toto ="""
'installation d ecs 34 9\xa0m²\r - ecs 1 : chauffe-eau standard  energie : electrique) avec accumulation horizontale 75\xa0l ; production en volume habitable  alimentant des pieces non contigues'
"""
toto ="""
poele au charbon   insert installe apres 2001 avec label flamme verte  systeme individuel)<br  
"""
#toto = td005_ch.loc[td005_ch.td001_dpe_id==id_].to_dict(orient='records')[1]['valeur_renseignee']

toto ='chaudiere elec.'
toto = ' - thema as condens f as 12 tn'
toto = ' - chaudiere gaz standard depuis 2001'

In [ ]:
search_instruction = '((reseau de chaleur) OR (reseaux de chaleurs) OR (reseaux de chaleur))'
search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['ecs thermodynamique electrique(pac ou ballon)'])
search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['abscence ecs solaire'])

search_instruction = generate_instruction_from_list(gen_ch_search_dict_flat['chaudiere energie indetermine condensation'])


In [ ]:
search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['chaudiere bois exact'])

search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['production mixte indetermine'])
search_instruction = generate_instruction_from_list(gen_ch_search_dict_flat['poele ou insert bois'])

In [ ]:
search_instruction = generate_instruction_from_list(energie_dict['gaz'])
search_instruction = generate_instruction_from_list(installation_dict['collectif'])

In [ ]:
#search_instruction='((mixte) OR (combine) OR (chauffage AND ecs) OR (lie ) OR (combine ) OR (idem))'

In [ ]:
from generate_dpe_annexes_scripts.text_matching_dict import *

In [ ]:
search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['chaudiere gaz standard'])

In [ ]:
search_instruction

In [ ]:
gen_ecs_search_dict_flat

In [ ]:
toto

In [ ]:
index_name = 'dpe_extract_text'
search_body = {
    "query": {
        "query_string": {
            "query": search_instruction,
            "default_field": "text_to_analyze"
        },

    },

}

es_client = setup_es_client(index_name=index_name)
bulk(es_client, gendata(index_name,['id']*1000,[toto]*1000))

time.sleep(1)
a_dict=es_client.search(index=index_name, body=search_body,size=500000)


In [ ]:
len(a_dict['hits']['hits'])

In [ ]:
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

In [ ]:
bulk(es_client, gendata(index_name,['id'],[toto]))


In [ ]:
es_client

In [ ]:
a_dict

In [ ]:

s_all = search_from_search_dict(es_client,search_dict,index_name=index_name)

In [ ]:
m_ch = m

In [ ]:
id_=m_ch.vr_source_ft.str.contains('air')&m_ch.vr_source_ft.str.contains('eau')&m_ch.vr_source_ft.str.contains('pac')

In [ ]:
m_ch.loc[m_ch.gen_ch_lib_ft.fillna('na').str.contains('air/eau')]

In [ ]:
m_ch.gen_ch_lib_<Q<

In [ ]:
s=m_ch.loc[m_ch.nom_methode_dpe_norm=='FACTURE'].gen_ch_lib_ft.fillna('indetermine').value_counts()

In [ ]:
s.loc['indetermine']/s.sum()

In [ ]:
(s=='indetermine').mean()

In [ ]:
m_ch.groupby('nom_methode_dpe_norm').gen_ch_lib_ft.value_counts()

In [ ]:
m.type_installation_ch_ft.value_counts()

In [ ]:
m.loc[m.gen_ch_lib_ft=='indetermine'].head(470).to_dict(orient='records')